<a href="https://colab.research.google.com/github/frankwilliammendozayucra/AGENDA_DE_CONTACTOS/blob/main/ADA_proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from typing import Literal
import time
from collections import deque
import traceback
import heapq
# Configuración para visualización
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

In [ ]:
# ========================================
# ALGORITMOS DE ORDENAMIENTO
# ========================================

class SortingAlgorithms:
    """Clase que implementa 10 algoritmos de ordenamiento"""

    @staticmethod
    def bubble_sort(df, column, ascending=True):
        """Bubble Sort - O(n²)"""
        df_copy = df.copy().reset_index(drop=True)
        n = len(df_copy)
        for i in range(n):
            for j in range(0, n-i-1):
                if ascending:
                    if df_copy.loc[j, column] > df_copy.loc[j+1, column]:
                        df_copy.iloc[j], df_copy.iloc[j+1] = df_copy.iloc[j+1].copy(), df_copy.iloc[j].copy()
                else: # Descending
                    if df_copy.loc[j, column] < df_copy.loc[j+1, column]:
                        df_copy.iloc[j], df_copy.iloc[j+1] = df_copy.iloc[j+1].copy(), df_copy.iloc[j].copy()
        return df_copy

    @staticmethod
    def selection_sort(df, column, ascending=True):
        """Selection Sort - O(n²)"""
        df_copy = df.copy().reset_index(drop=True)
        n = len(df_copy)
        for i in range(n):
            idx = i
            for j in range(i+1, n):
                if ascending:
                    if df_copy.loc[j, column] < df_copy.loc[idx, column]:
                        idx = j
                else: # Descending
                    if df_copy.loc[j, column] > df_copy.loc[idx, column]:
                        idx = j
            df_copy.iloc[i], df_copy.iloc[idx] = df_copy.iloc[idx].copy(), df_copy.iloc[i].copy()
        return df_copy

    @staticmethod
    def insertion_sort(df, column, ascending=True):
        """Insertion Sort - O(n²)"""
        df_copy = df.copy().reset_index(drop=True)
        for i in range(1, len(df_copy)):
            key_row = df_copy.iloc[i].copy()
            key_value = df_copy.loc[i, column]
            j = i - 1
            if ascending:
                while j >= 0 and df_copy.loc[j, column] > key_value:
                    df_copy.iloc[j + 1] = df_copy.iloc[j].copy()
                    j -= 1
            else: # Descending
                while j >= 0 and df_copy.loc[j, column] < key_value:
                    df_copy.iloc[j + 1] = df_copy.iloc[j].copy()
                    j -= 1
            df_copy.iloc[j + 1] = key_row
        return df_copy

    @staticmethod
    def merge_sort(df, column, ascending=True):
        """Merge Sort - O(n log n)"""
        if len(df) <= 1:
            return df.copy()

        mid = len(df) // 2
        left_half = df.iloc[:mid]
        right_half = df.iloc[mid:]

        left = SortingAlgorithms.merge_sort(left_half, column, ascending)
        right = SortingAlgorithms.merge_sort(right_half, column, ascending)

        return SortingAlgorithms._merge(left, right, column, ascending)

    @staticmethod
    def _merge(left, right, column, ascending):
        """Función auxiliar para merge sort"""
        result = []
        i = j = 0
        left_records = left.to_dict('records')
        right_records = right.to_dict('records')

        while i < len(left_records) and j < len(right_records):
            if ascending:
                if left_records[i][column] <= right_records[j][column]:
                    result.append(left_records[i])
                    i += 1
                else:
                    result.append(right_records[j])
                    j += 1
            else: # Descending
                if left_records[i][column] >= right_records[j][column]:
                    result.append(left_records[i])
                    i += 1
                else:
                    result.append(right_records[j])
                    j += 1

        result.extend(left_records[i:])
        result.extend(right_records[j:])

        return pd.DataFrame(result)

    @staticmethod
    def quick_sort(df, column, ascending=True):
        """Quick Sort - O(n log n) promedio"""
        if len(df) <= 1:
            return df.copy()

        pivot_row = df.iloc[len(df) // 2]
        pivot_value = pivot_row[column]

        if ascending:
            left = df[df[column] < pivot_value]
            middle = df[df[column] == pivot_value]
            right = df[df[column] > pivot_value]
        else: # Descending
            left = df[df[column] > pivot_value]
            middle = df[df[column] == pivot_value]
            right = df[df[column] < pivot_value]

        return pd.concat([
            SortingAlgorithms.quick_sort(left, column, ascending),
            middle,
            SortingAlgorithms.quick_sort(right, column, ascending)
        ]).reset_index(drop=True)

    @staticmethod
    def heap_sort(df, column, ascending=True):
        """Heap Sort - O(n log n)"""
        df_copy = df.copy().reset_index(drop=True)
        n = len(df_copy)

        def heapify(df_local, n_local, i_local):
            largest_or_smallest = i_local
            left = 2 * i_local + 1
            right = 2 * i_local + 2

            if ascending:
                if left < n_local and df_local.loc[i_local, column] < df_local.loc[left, column]:
                    largest_or_smallest = left
                if right < n_local and df_local.loc[largest_or_smallest, column] < df_local.loc[right, column]:
                    largest_or_smallest = right
            else: # Descending
                if left < n_local and df_local.loc[i_local, column] > df_local.loc[left, column]:
                    largest_or_smallest = left
                if right < n_local and df_local.loc[largest_or_smallest, column] > df_local.loc[right, column]:
                    largest_or_smallest = right

            if largest_or_smallest != i_local:
                df_local.iloc[i_local], df_local.iloc[largest_or_smallest] = df_local.iloc[largest_or_smallest].copy(), df_local.iloc[i_local].copy()
                heapify(df_local, n_local, largest_or_smallest)

        for i in range(n // 2 - 1, -1, -1):
            heapify(df_copy, n, i)

        for i in range(n - 1, 0, -1):
            df_copy.iloc[i], df_copy.iloc[0] = df_copy.iloc[0].copy(), df_copy.iloc[i].copy()
            heapify(df_copy, i, 0)

        return df_copy

    @staticmethod
    def shell_sort(df, column, ascending=True):
        """Shell Sort - Complejidad variable, mejor que O(n²)"""
        df_copy = df.copy().reset_index(drop=True)
        n = len(df_copy)
        gap = n // 2
        while gap > 0:
            for i in range(gap, n):
                temp_row = df_copy.iloc[i].copy()
                temp_val = temp_row[column]
                j = i
                if ascending:
                    while j >= gap and df_copy.loc[j - gap, column] > temp_val:
                        df_copy.iloc[j] = df_copy.iloc[j - gap].copy()
                        j -= gap
                else: # Descending
                    while j >= gap and df_copy.loc[j - gap, column] < temp_val:
                        df_copy.iloc[j] = df_copy.iloc[j - gap].copy()
                        j -= gap
                df_copy.iloc[j] = temp_row
            gap //= 2
        return df_copy

    @staticmethod
    def counting_sort(df, column, ascending=True):
        """Counting Sort - O(n + k) - Solo para enteros no negativos"""
        if not pd.api.types.is_integer_dtype(df[column]) or (df[column] < 0).any():
            raise ValueError("Counting Sort solo funciona con enteros no negativos.")

        df_copy = df.copy()
        max_val = df_copy[column].max()
        count = np.zeros(max_val + 1, dtype=int)

        for val in df_copy[column]:
            count[val] += 1

        for i in range(1, len(count)):
            count[i] += count[i-1]

        output = [None] * len(df_copy)
        for i in range(len(df_copy) - 1, -1, -1):
            row = df_copy.iloc[i]
            val = row[column]
            output[count[val] - 1] = row
            count[val] -= 1

        result_df = pd.DataFrame(output)
        if not ascending:
            result_df = result_df.iloc[::-1]

        return result_df.reset_index(drop=True)

    @staticmethod
    def radix_sort(df, column, ascending=True):
        """Radix Sort - O(d * (n + k)) - Solo para enteros no negativos"""
        if not pd.api.types.is_integer_dtype(df[column]) or (df[column] < 0).any():
             raise ValueError("Radix Sort solo funciona con enteros no negativos.")

        df_copy = df.copy()
        max_val = df_copy[column].max()
        exp = 1
        while max_val // exp > 0:
            df_copy = SortingAlgorithms._counting_sort_for_radix(df_copy, column, exp)
            exp *= 10

        if not ascending:
            df_copy = df_copy.iloc[::-1]

        return df_copy.reset_index(drop=True)

    @staticmethod
    def _counting_sort_for_radix(df, column, exp):
        n = len(df)
        output = [None] * n
        count = [0] * 10

        for i in range(n):
            index = df.loc[i, column] // exp
            count[index % 10] += 1

        for i in range(1, 10):
            count[i] += count[i - 1]

        i = n - 1
        while i >= 0:
            index = df.loc[i, column] // exp
            output[count[index % 10] - 1] = df.iloc[i].to_dict()
            count[index % 10] -= 1
            i -= 1

        return pd.DataFrame(output)

    @staticmethod
    def tim_sort(df, column, ascending=True):
        """Tim Sort (implementación de Pandas) - O(n log n)"""
        # kind='mergesort' o 'stable' es similar a Timsort
        return df.sort_values(by=column, ascending=ascending, kind='stable').reset_index(drop=True)

In [ ]:
# ========================================
# FUNCIÓN DE PARTICIONAMIENTO MEJORADA
# ========================================

def df_partition(df, criterion, percentage=0.25, partition_type="best",
                 sorting_algorithm="quick_sort"):
    """
    Segmenta un DataFrame según un criterio usando un algoritmo de ordenamiento específico.

    Parámetros:
    - df: DataFrame de entrada
    - criterion: Columna para ordenar
    - percentage: Porcentaje del segmento (0.0 a 1.0)
    - partition_type: "best" (valores más altos) o "worst" (valores más bajos)
    - sorting_algorithm: Nombre del algoritmo a usar de la clase SortingAlgorithms
    """
    if not 0 < percentage <= 1:
        raise ValueError("El porcentaje debe estar entre 0 y 1")

    sorter = SortingAlgorithms()
    sort_method = getattr(sorter, sorting_algorithm, None)

    if sort_method is None:
        raise ValueError(f"Algoritmo '{sorting_algorithm}' no disponible en SortingAlgorithms")

    # Para "best" queremos orden descendente, para "worst" ascendente.
    # Pero para tomar el primer segmento, siempre ordenamos y tomamos el inicio.
    # La clave es el orden de clasificación.
    # Best (más altos) -> descending=False -> ascending=False
    # Worst (más bajos) -> ascending=True
    ascending_order = (partition_type == "worst")

    start_time = time.time()

    # El orden descendente se logra con ascending=False
    df_sorted = sort_method(df, criterion, ascending=not ascending_order)

    partition_size = int(len(df) * percentage)

    # Tomamos el primer 'partition_size' de filas, que serán las mejores o peores.
    df_result = df_sorted.iloc[:partition_size].copy()

    end_time = time.time()
    execution_time = end_time - start_time

    print(f"Algoritmo usado: {sorting_algorithm}")
    print(f"Tiempo de ejecución: {execution_time:.6f} segundos")
    print(f"Tamaño de la partición '{partition_type}': {len(df_result)} filas")

    return df_result, execution_time

In [ ]:
# ========================================
# PASO 1: GENERACIÓN DE DATOS
# ========================================
print("="*80)
print("PASO 1: GENERANDO DATASET DE EJEMPLO")
print("="*80)


np.random.seed(42)
n_rows = 10000

# Generate heterogeneous data with specified relationships and correlations
data = {}
data['x1'] = np.random.rand(n_rows) * 100
data['x2'] = np.random.rand(n_rows) * 100
data['x5'] = np.random.rand(n_rows) * 100
data['x7'] = np.random.rand(n_rows) * 100

# Variables with specific relationships and noise
x6 = np.random.rand(n_rows) * 0.1 + 0.9 * np.sin(np.arange(n_rows) / 500)
x9 = np.random.rand(n_rows) * 0.1 + 0.9 * np.sin(np.arange(n_rows) / 500)
x4 = np.random.randint(0, 100, n_rows)
x8 = np.random.rand(n_rows) * 100

# Introduce correlations for x3, x6, x9
cov_matrix_x3_x6_x9 = [[1, 0.75, 0.75],
                       [0.75, 1, 0.75],
                       [0.75, 0.75, 1]]
correlated_x3_x6_x9 = np.random.multivariate_normal([0, 0, 0], cov_matrix_x3_x6_x9, n_rows)
data['x3'] = correlated_x3_x6_x9[:, 0] * np.std(x6) + np.mean(x6)
data['x6'] = correlated_x3_x6_x9[:, 1] * np.std(x6) + np.mean(x6)
data['x9'] = correlated_x3_x6_x9[:, 2] * np.std(x9) + np.mean(x9)

# Introduce correlations for x10, x4, x6, x8
cov_matrix_x10_x4_x6_x8 = [[1, 0.85, 0.85, 0.85],
                           [0.85, 1, 0.85, 0.85],
                           [0.85, 0.85, 1, 0.85],
                           [0.85, 0.85, 0.85, 1]]
correlated_x10_x4_x6_x8 = np.random.multivariate_normal([0, 0, 0, 0], cov_matrix_x10_x4_x6_x8, n_rows)
data['x10'] = correlated_x10_x4_x6_x8[:, 0]
data['x4'] = correlated_x10_x4_x6_x8[:, 1] * np.std(x4) + np.mean(x4)
data['x6'] = correlated_x10_x4_x6_x8[:, 2] * np.std(x6) + np.mean(x6)
data['x8'] = correlated_x10_x4_x6_x8[:, 3] * np.std(x8) + np.mean(x8)

# Create the DataFrame
df = pd.DataFrame(data)

# ==========================================
# Crear la variable y = x3 * x10
# Con distribución gaussiana
# ==========================================

# Calcular el producto base
y_base = df['x3'] * df['x10']

# Normalizar a media 0 y desviación estándar 1
y_normalized = (y_base - y_base.mean()) / y_base.std()

# Añadir ruido gaussiano para mejorar la distribución normal
ruido_gaussiano = np.random.normal(0, 0.1, n_rows)  # Media 0, std pequeña
y = y_normalized + ruido_gaussiano



# Añadir la variable a la dataframe
df['y'] = y

print(f"\nDataset creado con {df.shape[0]} filas y {df.shape[1]} columnas.")
print("\nPrimeras 5 filas del dataset:")
display(df.head())
print("\nEstadísticas de la columna 'y':")
print(df['y'].describe())

In [ ]:
# ========================================
# PASO 2: COMPARAR ALGORITMOS Y CREAR PARTICIONES
# ========================================
print("\n" + "="*80)
print("PASO 2.1: COMPARANDO RENDIMIENTO DE ALGORITMOS DE ORDENAMIENTO")
print("="*80)

# Algoritmos eficientes a comparar
algorithms_to_compare = ['quick_sort', 'merge_sort']
for algo in algorithms_to_compare:
    print(f"\n--- Probando: {algo.replace('_', ' ').title()} ---")
    df_partition(df, 'y', percentage=0.25, partition_type="best", sorting_algorithm=algo)

print("\n\n" + "="*80)
print("PASO 2.2: CREANDO PARTICIONES CON EL ALGORITMO ELEGIDO (quick_sort)")
print("="*80)

# Seleccionamos quick_sort por su buen rendimiento general
selected_algorithm = 'quick_sort'
B4C, _ = df_partition(df, 'y', 0.25, "best", selected_algorithm)
W4C, _ = df_partition(df, 'y', 0.25, "worst", selected_algorithm)

print("\nPartición de los MEJORES 25% (B4C):")
display(B4C.head())
print("\nPartición de los PEORES 25% (W4C):")
display(W4C.head())

In [ ]:
# ========================================
# PASO 3: CREACIÓN Y ANÁLISIS DETALLADO DE PARTICIONES
# ========================================
print("\n" + "="*80)
print("PASO 3.1: CREANDO MÚLTIPLES PARTICIONES CON 'quick_sort'")
print("="*80)

selected_algorithm = 'quick_sort'

# Crear particiones
B2C, _ = df_partition(df, 'y', 0.50, "best", selected_algorithm)
W2C, _ = df_partition(df, 'y', 0.50, "worst", selected_algorithm)
B4C, _ = df_partition(df, 'y', 0.25, "best", selected_algorithm)
W4C, _ = df_partition(df, 'y', 0.25, "worst", selected_algorithm)
B8C, _ = df_partition(df, 'y', 0.125, "best", selected_algorithm)
W8C, _ = df_partition(df, 'y', 0.125, "worst", selected_algorithm)
B16C, _ = df_partition(df, 'y', 0.0625, "best", selected_algorithm)
W16C, _ = df_partition(df, 'y', 0.0625, "worst", selected_algorithm)

print("\nResumen de tamaños de las particiones:")
print(f"B2C (Best 50%): {B2C.shape}")
print(f"W2C (Worst 50%): {W2C.shape}")
print(f"B4C (Best 25%): {B4C.shape}")
print(f"W4C (Worst 25%): {W4C.shape}")
print(f"B8C (Best 12.5%): {B8C.shape}")
print(f"W8C (Worst 12.5%): {W8C.shape}")
print(f"B16C (Best 6.25%): {B16C.shape}")
print(f"W16C (Worst 6.25%): {W16C.shape}")

In [ ]:
# ========================================
# PASO 3.2: ANÁLISIS ESTADÍSTICO DE LAS PARTICIONES
# ========================================
print("\n" + "="*80)
print("PASO 3.2: ANÁLISIS ESTADÍSTICO DE LAS PARTICIONES")
print("="*80)

particiones = {
    'df_original': df, 'B2C': B2C, 'W2C': W2C, 'B4C': B4C, 'W4C': W4C,
    'B8C': B8C, 'W8C': W8C, 'B16C': B16C, 'W16C': W16C
}

estadisticas = []
for nombre, partition_df in particiones.items():
    stats = {
        'Partición': nombre,
        'Tamaño': len(partition_df),
        'Min_y': partition_df['y'].min(),
        'Max_y': partition_df['y'].max(),
        'Media_y': partition_df['y'].mean(),
        'Mediana_y': partition_df['y'].median(),
        'Std_Dev_y': partition_df['y'].std()
    }
    estadisticas.append(stats)

df_stats = pd.DataFrame(estadisticas).set_index('Partición')
display(df_stats.round(2))

In [ ]:
# ========================================
# PASO 3.3: VISUALIZACIÓN DE DENSIDAD
# ========================================
print("\n" + "="*80)
print("PASO 3.3: GRÁFICO DE DENSIDAD DE 'y' POR PARTICIÓN")
print("="*80)

plt.figure(figsize=(18, 8))
# Graficar solo las particiones para no saturar el gráfico
sns.kdeplot(data=df['y'], label='Original', fill=True, alpha=0.2, color='grey')
sns.kdeplot(data=B4C['y'], label='B4C (Mejores 25%)', fill=True, alpha=0.5)
sns.kdeplot(data=W4C['y'], label='W4C (Peores 25%)', fill=True, alpha=0.5)
sns.kdeplot(data=B16C['y'], label='B16C (Mejores 6.25%)', fill=True, alpha=0.5)
sns.kdeplot(data=W16C['y'], label='W16C (Peores 6.25%)', fill=True, alpha=0.5)
sns.kdeplot(data=B2C['y'], label='B2C (Mejores 50%)', fill=True, alpha=0.8)
sns.kdeplot(data=W2C['y'], label='W2C (Peores 50%)', fill=True, alpha=0.8)
sns.kdeplot(data=B8C['y'], label='B8C (Mejores 12.50%)', fill=True, alpha=0.3)
sns.kdeplot(data=W8C['y'], label='W8C (Peores 12.50%)', fill=True, alpha=0.3)


plt.title('Distribución de la Variable "y" en Diferentes Particiones', fontsize=16)
plt.xlabel('Valor de y', fontsize=12)
plt.ylabel('Densidad', fontsize=12)
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
# ========================================
# PASO 3.4: VISUALIZACIÓN DE DIAGRAMA DE CAJAS
# ========================================
print("\n" + "="*80)
print("PASO 3.4: DIAGRAMA DE CAJAS DE 'y' POR PARTICIÓN")
print("="*80)

# Preparar datos y etiquetas para el boxplot
data_to_plot = [B2C['y'], W2C['y'], B4C['y'], W4C['y'], B8C['y'], W8C['y'], B16C['y'], W16C['y']]
labels = ['B2C', 'W2C', 'B4C', 'W4C', 'B8C', 'W8C', 'B16C', 'W16C']

plt.figure(figsize=(16, 8))
plt.boxplot(data_to_plot, labels=labels, vert=False, patch_artist=True)
plt.title('Distribución de "y" por Partición', fontsize=16)
plt.xlabel('Valor de y', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# ========================================
# CORRELACIÓN DE PEARSON CON THRESHOLD FIJO
# Usuario define el threshold
# MUESTRA MATRIZ COMPLETA Y LA GRAFICA EN UN MAPA DE CALOR
# ========================================

import math
import pandas as pd
import matplotlib.pyplot as plt # <--- MODIFICACIÓN: Importar matplotlib
import seaborn as sns         # <--- MODIFICACIÓN: Importar seaborn

print("\n" + "="*120)
print("ANÁLISIS DE CORRELACIÓN DE PEARSON CON THRESHOLD FIJO (Y MATRIZ GRÁFICA)")
print("="*120)

# ⭐ PARÁMETRO CONFIGURABLE POR EL USUARIO ⭐
THRESHOLD = 0.01  # ← CAMBIA ESTE VALOR AQUÍ

print(f"\n📊 CONFIGURACIÓN:")
print(f"   Threshold: {THRESHOLD}")
print(f"   Solo correlaciones POSITIVAS r ≥ {THRESHOLD}\n")

# Diccionario para almacenar resultados
resultados_pearson = {}

# --- Asumiendo que estos DataFrames existen en tu entorno ---
# (Creando dummies para que el script sea ejecutable)
try:
    df, B2C, W2C, B4C, W4C, B8C, W8C, B16C, W16C
except NameError:
    print("ADVERTENCIA: Creando DataFrames de ejemplo...")
    data_ejemplo = {
        'VarA': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'VarB': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], # Correlación Fuerte con A
        'VarC': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1], # Correlación Fuerte Negativa con A
        'VarD': [1, 3, 2, 5, 4, 7, 6, 9, 8, 10], # Correlación Moderada
        'VarE': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # Cero varianza
        'VarF': [5, 6, 5, 6, 5, 6, 5, 6, 5, 6]  # Baja varianza
    }
    df = pd.DataFrame(data_ejemplo)
    B2C = df.head(5)
    W2C = df.tail(5)
    B4C = df.head(3)
    W4C = df.tail(3)
    B8C = df.head(8)
    W8C = df.tail(8)
    B16C = df.head(6)
    W16C = df.tail(6)
    print("...DataFrames de ejemplo creados.\n")
# -----------------------------------------------------------


# Particiones a analizar
particiones_analizar = {
    'df_original': df,
    'B2C': B2C,
    'W2C': W2C,
    'B4C': B4C,
    'W4C': W4C,
    'B8C': B8C,
    'W8C': W8C,
    'B16C': B16C,
    'W16C': W16C
}

# <--- INICIO MODIFICACIÓN: Función para generar el mapa de calor ---
def plot_correlation_heatmap(correlation_df, partition_name, threshold=None):
    if correlation_df.empty:
        print(f"  No se puede generar mapa de calor para {partition_name}: DataFrame de correlación vacío.")
        return

    plt.figure(figsize=(len(correlation_df.columns)*0.8, len(correlation_df.index)*0.8)) # Tamaño dinámico

    title = f'Mapa de Calor de Correlación de Pearson - {partition_name}'
    if threshold is not None:
        title += f' (Threshold r ≥ {threshold})'

    sns.heatmap(
        correlation_df,
        annot=True,          # Mostrar los valores de correlación en el mapa
        cmap='coolwarm',     # Esquema de color (rojo para negativo, azul para positivo)
        fmt=".2f",           # Formato de los números a dos decimales
        linewidths=.5,       # Líneas entre las celdas
        vmin=-1, vmax=1      # Asegurar que la escala de color va de -1 a 1
    )
    plt.title(title, fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout() # Ajusta el diseño para que todo quepa

    filename = f'pearson_heatmap_{partition_name}.png'
    plt.savefig(filename)
    plt.close() # Cierra la figura para liberar memoria
    print(f"  ✅ Mapa de calor guardado: {filename}")
# <--- FIN MODIFICACIÓN ---


# ========================================
# LOOP PRINCIPAL - ANALIZAR CADA PARTICIÓN
# ========================================

for nombre_particion, partition_df in particiones_analizar.items():

    n_datos = len(partition_df)

    print(f"\n{'='*120}")
    print(f"PARTICIÓN: {nombre_particion.upper()} ({n_datos} filas)")
    print(f"{'='*120}\n")

    # 1. Obtener columnas numéricas
    numerical_cols = []
    if n_datos == 0:
        print("1. DataFrame vacío (0 filas).")
        resultados_pearson[nombre_particion] = {'n_datos': 0, 'threshold': THRESHOLD, 'aristas': [], 'num_aristas': 0, 'matriz_completa': pd.DataFrame()}
        continue # Saltar al siguiente partición

    for col in partition_df.columns:
        try:
            # Intentar convertir el primer valor (ignorar si está vacío)
            # También considerar el caso donde toda la columna es NaN o no-numérica después de filtrado
            if pd.to_numeric(partition_df[col], errors='coerce').notna().any(): # Verifica si hay algún valor numérico válido
                numerical_cols.append(col)
        except (ValueError, TypeError, IndexError):
            pass

    if not numerical_cols:
        print("1. No se encontraron columnas numéricas o el DataFrame es demasiado pequeño.")
        resultados_pearson[nombre_particion] = {'n_datos': n_datos, 'threshold': THRESHOLD, 'aristas': [], 'num_aristas': 0, 'matriz_completa': pd.DataFrame()}
        continue

    print(f"1. Variables numéricas: {numerical_cols}")

    # Crear la matriz de correlación (DataFrame vacío)
    corr_matrix_df = pd.DataFrame(index=numerical_cols, columns=numerical_cols, dtype=float)

    # 2. Calcular correlaciones de Pearson
    print(f"\n2. Calculando correlaciones de Pearson...\n")

    aristas_significativas = []

    # Rellenar la matriz de correlación completa
    for i in range(len(numerical_cols)):
        col1_name = numerical_cols[i]
        corr_matrix_df.loc[col1_name, col1_name] = 1.0

        for j in range(i + 1, len(numerical_cols)):
            col2_name = numerical_cols[j]

            # Obtener valores (filtrando NaNs si los hay)
            # Usamos pd.to_numeric para manejar no-numéricos y luego dropear NaNs
            series1 = pd.to_numeric(partition_df[col1_name], errors='coerce').dropna()
            series2 = pd.to_numeric(partition_df[col2_name], errors='coerce').dropna()

            # Asegurarse de que ambas series tienen los mismos índices para un cálculo correcto
            # Si se espera que los DataFrames no tengan NaNs en las columnas numéricas, esto es redundante
            # Pero es una buena práctica para robustez
            common_index = series1.index.intersection(series2.index)

            col1_vals = series1.loc[common_index].tolist()
            col2_vals = series2.loc[common_index].tolist()

            n_valid_datos = len(col1_vals) # Número de datos válidos para este par

            r = 0.0 # Valor por defecto si no hay suficientes datos
            if n_valid_datos > 1: # Se necesitan al menos 2 puntos para calcular correlación
                media_x = sum(col1_vals) / n_valid_datos
                media_y = sum(col2_vals) / n_valid_datos

                suma_xy = sum((col1_vals[k] - media_x) * (col2_vals[k] - media_y) for k in range(n_valid_datos))
                suma_x2 = sum((col1_vals[k] - media_x) ** 2 for k in range(n_valid_datos))
                suma_y2 = sum((col2_vals[k] - media_y) ** 2 for k in range(n_valid_datos))

                denom = math.sqrt(suma_x2 * suma_y2)
                if denom != 0:
                    r = suma_xy / denom
                # else r = 0.0 (ya inicializado)
            # else r = 0.0 (ya inicializado)

            # Guardar en la matriz (antes de filtrar)
            corr_matrix_df.loc[col1_name, col2_name] = r
            corr_matrix_df.loc[col2_name, col1_name] = r

            # FILTRAR: Solo positivas Y mayor o igual al threshold
            if r >= THRESHOLD:
                aristas_significativas.append({
                    'var1': col1_name,
                    'var2': col2_name,
                    'r_pearson': r
                })

    # 2b. MOSTRAR LA MATRIZ COMPLETA (en texto)
    if not corr_matrix_df.empty:
        print(f"2b. Matriz de Correlación de Pearson COMPLETA:\n")
        print(corr_matrix_df.to_string(float_format="%.6f"))
        print("\n" + "-"*50 + "\n")

        # <--- INICIO MODIFICACIÓN: Generar y mostrar el mapa de calor ---
        print("  Generando mapa de calor...")
        plot_correlation_heatmap(corr_matrix_df, nombre_particion)
        # --- FIN MODIFICACIÓN ---

    else:
        print("2b. No se pudo generar una matriz de correlación (no hay columnas numéricas válidas).")

    # Ordenar por correlación descendente
    aristas_significativas.sort(key=lambda x: x['r_pearson'], reverse=True)

    num_aristas = len(aristas_significativas)
    print(f"3. Aristas encontradas (r ≥ {THRESHOLD}): {num_aristas}\n")

    # Estadísticas
    if len(aristas_significativas) > 0:
        r_max = max([a['r_pearson'] for a in aristas_significativas])
        r_min = min([a['r_pearson'] for a in aristas_significativas])
        r_prom = sum([a['r_pearson'] for a in aristas_significativas]) / len(aristas_significativas)

        print(f"4. Estadísticas (filtradas):")
        print(f"   Correlación máxima: {r_max:.6f}")
        print(f"   Correlación mínima: {r_min:.6f}")
        print(f"   Correlación promedio: {r_prom:.6f}\n")

        # Mostrar todas o top 15
        num_mostrar = min(15, len(aristas_significativas))
        print(f"5. Top {num_mostrar} correlaciones (filtradas):\n")
        print(f"   {'Nº':<4} {'Variable 1':<12} {'Variable 2':<12} {'Pearson r':<15}")
        print(f"   {'-'*50}")

        for idx, arista in enumerate(aristas_significativas[:num_mostrar], 1):
            print(f"   {idx:<4} {arista['var1']:<12} {arista['var2']:<12} {arista['r_pearson']:>+.8f}")

        if len(aristas_significativas) > 15:
            print(f"\n   ... y {len(aristas_significativas) - 15} más")
    else:
        print(f" 4. ⚠️   Sin correlaciones positivas por encima del threshold ({THRESHOLD})\n")

    # Guardar resultados
    resultados_pearson[nombre_particion] = {
        'n_datos': n_datos,
        'threshold': THRESHOLD,
        'aristas': aristas_significativas,
        'num_aristas': num_aristas,
        'matriz_completa': corr_matrix_df
    }


# ========================================
# TABLA COMPARATIVA GENERAL
# ========================================

print(f"\n\n{'='*120}")
print("TABLA COMPARATIVA: RESULTADOS POR PARTICIÓN")
print(f"{'='*120}\n")

print(f"Threshold aplicado: {THRESHOLD}\n")

print(f"{'Partición':<15} {'Filas':<10} {'Aristas':<12} {'r_Máx':<15} {'r_Mín':<15} {'r_Prom':<15} {'Ratio':<12}")
print("-"*120)

for nombre, datos in resultados_pearson.items():
    n = datos['n_datos']
    num_aristas = datos['num_aristas']

    if len(datos['aristas']) > 0:
        r_max = max([a['r_pearson'] for a in datos['aristas']])
        r_min = min([a['r_pearson'] for a in datos['aristas']])
        r_prom = sum([a['r_pearson'] for a in datos['aristas']]) / len(datos['aristas'])
        ratio = num_aristas / n if n > 0 else 0
    else:
        r_max = 0
        r_min = 0
        r_prom = 0
        ratio = 0

    print(f"{nombre:<15} {n:<10} {num_aristas:<12} {r_max:>+.8f}    "
          f"{r_min:>+.8f}    {r_prom:>+.8f}    {ratio:.6f}")

print("="*120)


# ========================================
# GRÁFICO COMPARATIVO ASCII
# ========================================

print(f"\n\n{'='*120}")
print("VISUALIZACIÓN: NÚMERO DE ARISTAS POR PARTICIÓN")
print(f"{'='*120}\n")

max_aristas = max([datos['num_aristas'] for datos in resultados_pearson.values()] + [0]) # +[0] para evitar error si todo está vacío

for nombre, datos in resultados_pearson.items():
    num_aristas = datos['num_aristas']

    if max_aristas > 0:
        barra_len = int((num_aristas / max_aristas) * 50)
    else:
        barra_len = 0

    barra = "█" * barra_len + "░" * (50 - barra_len)

    print(f"{nombre:<15} │{barra}│ {num_aristas:>3} aristas")

print("\n" + "="*120)


# ========================================
# ANÁLISIS POR RANGO DE CORRELACIÓN
# ========================================

print(f"\n{'='*120}")
print("DISTRIBUCIÓN POR RANGOS DE CORRELACIÓN (Sobre aristas filtradas)")
print(f"{'='*120}\n")

rangos = {
    f'Muy fuerte (r ≥ 0.9)': (0.9, 1.0),
    f'Fuerte (0.8 ≤ r < 0.9)': (0.8, 0.9),
    f'Moderada (0.7 ≤ r < 0.8)': (0.7, 0.8),
    f'Débil ({THRESHOLD} ≤ r < 0.7)': (THRESHOLD, 0.7),
}

for nombre, datos in resultados_pearson.items():
    print(f"\n{nombre.upper()}:")

    if len(datos['aristas']) > 0:
        total_aristas = len(datos['aristas'])
        # Ajuste para incluir r = 1.0 en el rango "Muy fuerte"
        rangos[f'Muy fuerte (r ≥ 0.9)'] = (0.9, 1.01)

        for rango_nombre, (min_r, max_r) in rangos.items():
            count = sum(1 for a in datos['aristas'] if min_r <= a['r_pearson'] < max_r or (rango_nombre.startswith('Muy') and a['r_pearson'] == 1.0))
            porcent = (count / total_aristas * 100) if total_aristas > 0 else 0

            # Escalar barra para que no sea tan larga
            barra_len_rango = int(porcent / 5) # 1 '█' por cada 5%
            barra = "█" * barra_len_rango

            print(f"   {rango_nombre:<35} {barra:<20} {count:>3} ({porcent:.1f}%)")
    else:
        print(f"   Sin aristas")

print("\n" + "="*120)


# ========================================
# EXPORTAR RESULTADOS
# ========================================

print(f"\n{'='*120}")
print("GUARDANDO RESULTADOS")
print(f"{'='*120}\n")

# Guardar tabla resumida
with open('pearson_resumen.csv', 'w', encoding='utf-8') as f:
    f.write(f"threshold,{THRESHOLD}\n")
    f.write("Partición,Filas,Aristas,r_Max,r_Min,r_Promedio\n")

    for nombre, datos in resultados_pearson.items():
        n = datos['n_datos']
        num_aristas = datos['num_aristas']

        if len(datos['aristas']) > 0:
            r_max = max([a['r_pearson'] for a in datos['aristas']])
            r_min = min([a['r_pearson'] for a in datos['aristas']])
            r_prom = sum([a['r_pearson'] for a in datos['aristas']]) / len(datos['aristas'])
        else:
            r_max = 0
            r_min = 0
            r_prom = 0

        f.write(f"{nombre},{n},{num_aristas},{r_max:.8f},{r_min:.8f},{r_prom:.8f}\n")

print("✅ Resumen guardado: pearson_resumen.csv")

# Guardar detalles de aristas por partición
for nombre, datos in resultados_pearson.items():
    filename_aristas = f"pearson_aristas_{nombre}.csv"
    with open(filename_aristas, 'w', encoding='utf-8') as f:
        f.write("var1,var2,pearson_r\n")
        for arista in datos['aristas']:
            f.write(f"{arista['var1']},{arista['var2']},{arista['r_pearson']:.8f}\n")

    if len(datos['aristas']) > 0:
        print(f"✅ Aristas guardadas: {filename_aristas} ({len(datos['aristas'])} correlaciones)")
    else:
        print(f"⚠️   Sin aristas para: {nombre} (archivo vacío creado)")

    # Guardar la matriz de correlación completa
    filename_matriz = f"pearson_matriz_completa_{nombre}.csv"
    if not datos['matriz_completa'].empty:
        datos['matriz_completa'].to_csv(filename_matriz, encoding='utf-8', float_format='%.8f')
        print(f"✅ Matriz completa guardada: {filename_matriz}")
    else:
        print(f"⚠️   No hay matriz completa para guardar para: {nombre}")


print(f"\n{'='*120}")
print("ANÁLISIS COMPLETADO")
print(f"{'='*120}\n")

In [ ]:

# ========================================
# ALGORITMOS DE GRAFOS
# ========================================

class GraphAlgorithms:
    """Contiene implementaciones para MST (Kruskal y Prim)."""

    class DSU:
        """Helper class for Disjoint Set Union (DSU), used by Kruskal."""
        def __init__(self, nodes):
            self.parent = {node: node for node in nodes}

        def find(self, i):
            if self.parent[i] == i:
                return i
            self.parent[i] = self.find(self.parent[i])
            return self.parent[i]

        def union(self, x, y):
            root_x = self.find(x)
            root_y = self.find(y)
            if root_x != root_y:
                self.parent[root_x] = root_y
                return True
            return False

    @staticmethod
    def kruskal_mst(graph):
        """Calcula el MST usando Kruskal."""
        mst = nx.Graph()
        mst.add_nodes_from(graph.nodes())
        total_weight = 0

        edges = sorted(graph.edges(data=True), key=lambda t: t[2].get('weight', 1))

        dsu = GraphAlgorithms.DSU(graph.nodes())

        for u, v, data in edges:
            if dsu.union(u, v):
                mst.add_edge(u, v, **data)
                total_weight += data.get('weight', 1)

        return mst, total_weight

    @staticmethod
    def prim_mst(graph, start_node=None):
        """Calcula el MST usando Prim."""
        mst = nx.Graph()
        total_weight = 0

        if not graph.nodes():
            return mst, total_weight

        if start_node is None or start_node not in graph:
            valid_start_node = None
            for node in graph.nodes():
                if graph.degree(node) > 0:
                    valid_start_node = node
                    break
            if valid_start_node is None:
                if not graph.nodes():
                    return mst, total_weight
                valid_start_node = list(graph.nodes())[0]
            start_node = valid_start_node

        if start_node not in graph:
             return mst, total_weight

        mst.add_node(start_node)
        priority_queue = []

        for neighbor in graph.neighbors(start_node):
            weight = graph[start_node][neighbor].get('weight', 1)
            heapq.heappush(priority_queue, (weight, start_node, neighbor))

        while priority_queue and mst.number_of_nodes() < graph.number_of_nodes():
            weight, u, v = heapq.heappop(priority_queue)

            if v not in mst.nodes():
                mst.add_node(v)

                edge_data = graph[u][v].copy()
                edge_data.pop('weight', None)

                mst.add_edge(u, v, weight=weight, **edge_data)
                total_weight += weight

                for neighbor in graph.neighbors(v):
                    if neighbor not in mst.nodes():
                        neighbor_weight = graph[v][neighbor].get('weight', 1)
                        heapq.heappush(priority_queue, (neighbor_weight, v, neighbor))

        for node in graph.nodes():
            if node not in mst.nodes():
                mst.add_node(node)

        return mst, total_weight


# ========================================
# FUNCIÓN DE VISUALIZACIÓN CORREGIDA
# ========================================

def visualize_graphs_and_mst(aristas_filtradas, numerical_cols, partition_name, threshold):
    """
    Visualiza Grafo Original, MST Kruskal y MST Prim en configuración 1x3.
    VERSIÓN CORREGIDA: Sin zorder en draw_networkx_nodes()
    """
    print(f"\n--- Generando Grafos y MSTs para: {partition_name} ---")

    G = nx.Graph()
    G.add_nodes_from(numerical_cols)

    if not aristas_filtradas:
        print(f"  ⚠️  Sin aristas significativas (r >= {threshold}). El grafo no tendrá aristas.")
    else:
        for arista in aristas_filtradas:
            r = arista['r_pearson']
            weight = 1.0 - r
            G.add_edge(arista['var1'], arista['var2'], weight=weight, correlation=r)

    # Calcular MSTs
    mst_kruskal, peso_k = GraphAlgorithms.kruskal_mst(G)
    mst_prim, peso_p = GraphAlgorithms.prim_mst(G)

    print(f"  ✓ Grafo Original: {G.number_of_nodes()} nodos, {G.number_of_edges()} aristas")
    print(f"  ✓ MST Kruskal: {mst_kruskal.number_of_nodes()} nodos, {mst_kruskal.number_of_edges()} aristas, Peso: {peso_k:.4f}")
    print(f"  ✓ MST Prim: {mst_prim.number_of_nodes()} nodos, {mst_prim.number_of_edges()} aristas, Peso: {peso_p:.4f}")

    # Crear figura 1x3
    fig, axes = plt.subplots(1, 3, figsize=(22, 8))
    fig.suptitle(f"Análisis de Grafo: {partition_name} (r >= {threshold})", fontsize=16, fontweight='bold')

    # Calcular layout
    k_layout = 1.5 / math.sqrt(max(1, G.number_of_nodes()))
    pos = nx.spring_layout(G, seed=42, k=k_layout, iterations=50)

    # Constantes de estilo
    node_size = 2000
    node_alpha = 0.85
    font_size = 10

    # ==========================================
    # 1. GRAFO ORIGINAL
    # ==========================================
    ax1 = axes[0]

    # Dibujar aristas con grosor dinámico
    if G.number_of_edges() > 0:
        edge_correlations = [d['correlation'] for u, v, d in G.edges(data=True)]
        max_width = 4.0
        min_width = 0.5
        range_denominator = (1.0 - threshold) if (1.0 - threshold) > 0 else 1.0
        edge_widths = [
            min_width + (corr - threshold) * (max_width - min_width) / range_denominator
            for corr in edge_correlations
        ]

        nx.draw_networkx_edges(
            G, pos, ax=ax1,
            edgelist=G.edges(),
            width=edge_widths,
            edge_color=edge_correlations,
            edge_cmap=plt.cm.Greens,
            edge_vmin=threshold, edge_vmax=1.0,
            alpha=0.7
        )

        # Etiquetas de aristas
        edge_labels = {
            (u, v): f"{d['correlation']:.2f}"
            for u, v, d in G.edges(data=True)
        }
        nx.draw_networkx_edge_labels(
            G, pos, ax=ax1,
            edge_labels=edge_labels,
            font_size=font_size - 2,
            font_color='black'
        )

    # Dibujar nodos (SIN zorder)
    nx.draw_networkx_nodes(
        G, pos, ax=ax1, node_color='skyblue',
        node_size=node_size, alpha=node_alpha,
        edgecolors='black', linewidths=2
    )

    # Etiquetas de nodos
    nx.draw_networkx_labels(
        G, pos, ax=ax1,
        font_size=font_size, font_weight='bold'
    )

    ax1.set_title(f"Grafo de Correlación Original\n({G.number_of_edges()} aristas)",
                  fontsize=12, fontweight='bold', pad=10)
    ax1.axis('off')

    # ==========================================
    # 2. MST KRUSKAL
    # ==========================================
    ax2 = axes[1]

    # Dibujar aristas
    nx.draw_networkx_edges(
        mst_kruskal, pos, ax=ax2,
        width=3, alpha=0.8, edge_color='darkgreen'
    )

    # Dibujar nodos (SIN zorder)
    nx.draw_networkx_nodes(
        mst_kruskal, pos, ax=ax2, node_color='lightgreen',
        node_size=node_size, alpha=node_alpha,
        edgecolors='darkgreen', linewidths=2
    )

    # Etiquetas
    nx.draw_networkx_labels(
        mst_kruskal, pos, ax=ax2,
        font_size=font_size, font_weight='bold'
    )

    ax2.set_title(f"Árbol Expansión Mínima (Kruskal)\n({mst_kruskal.number_of_edges()} aristas)",
                  fontsize=12, fontweight='bold', pad=10)
    ax2.axis('off')

    # ==========================================
    # 3. MST PRIM
    # ==========================================
    ax3 = axes[2]

    # Dibujar aristas
    nx.draw_networkx_edges(
        mst_prim, pos, ax=ax3,
        width=3, alpha=0.8, edge_color='darkred'
    )

    # Dibujar nodos (SIN zorder)
    nx.draw_networkx_nodes(
        mst_prim, pos, ax=ax3, node_color='lightcoral',
        node_size=node_size, alpha=node_alpha,
        edgecolors='darkred', linewidths=2
    )

    # Etiquetas
    nx.draw_networkx_labels(
        mst_prim, pos, ax=ax3,
        font_size=font_size, font_weight='bold'
    )

    ax3.set_title(f"Árbol Expansión Mínima (Prim)\n({mst_prim.number_of_edges()} aristas)",
                  fontsize=12, fontweight='bold', pad=10)
    ax3.axis('off')

    plt.tight_layout()
    plt.show()

    print(f"  ✓ Gráficos para {partition_name} mostrados.\n")


# ========================================
# FUNCIÓN PRINCIPAL
# ========================================

def mostrar_grafos_de_resultados():
    """Genera visualizaciones para todas las particiones."""
    print("\n" + "="*90)
    print("VISUALIZACIÓN DE GRAFOS: ORIGINAL, KRUSKAL Y PRIM")
    print("="*90)

    try:
        # Verificar si resultados_pearson existe
        if 'resultados_pearson' not in locals() and 'resultados_pearson' not in globals():
             print("\n⚠️ ERROR: La variable 'resultados_pearson' no se encontró.")
             print("Por favor, ejecuta primero el script de análisis de correlación.")
             return

        # Acceder a resultados_pearson
        global resultados_pearson

        if not resultados_pearson:
             print("\n⚠️ ADVERTENCIA: 'resultados_pearson' está vacía.")
             return

        print(f"\n✓ Se encontraron {len(resultados_pearson)} particiones.\n")

        # Iterar sobre resultados
        for partition_name, data in resultados_pearson.items():

            aristas = data.get('aristas', [])
            threshold = data.get('threshold', 0.01)

            # Obtener columnas numéricas
            # Asumimos que están en la partición original
            try:
                partition_df = particiones_analizar.get(partition_name)
                if partition_df is None:
                    print(f"⚠️  Saltando {partition_name}: No se encuentra en particiones_analizar")
                    continue

                numerical_cols = []
                for col in partition_df.columns:
                    try:
                        _ = float(partition_df[col].iloc[0])
                        numerical_cols.append(col)
                    except:
                        pass
            except:
                print(f"⚠️  Saltando {partition_name}: Error obteniendo columnas numéricas")
                continue

            # Visualizar
            visualize_graphs_and_mst(aristas, numerical_cols, partition_name, threshold)

        print("="*90)
        print("✅ VISUALIZACIÓN DE GRAFOS COMPLETADA")
        print("="*90)

    except NameError as e:
        print(f"\n❌ ERROR: {e}")
        print("Asegúrate de ejecutar el script de análisis de correlación primero.")
    except Exception as e:
        print(f"\n❌ Error inesperado: {e}")
        traceback.print_exc()


# Ejecutar
mostrar_grafos_de_resultados()

In [ ]:
# ========================================
# ALGORITMO DE NEWMAN - SOLO IMPRIME RESULTADOS
# ========================================

import networkx as nx


class NewmanAlgorithm:
    """Detección de comunidades - Algoritmo de Newman"""

    def __init__(self, grafo_nx, nombre_particion=""):
        self.grafo = grafo_nx
        self.nombre = nombre_particion
        self.nodos = list(grafo_nx.nodes())
        self.n = len(self.nodos)
        self.nodo_a_idx = {nodo: idx for idx, nodo in enumerate(self.nodos)}
        self.idx_a_nodo = {idx: nodo for nodo, idx in self.nodo_a_idx.items()}

    def _inicializar_modularidad(self):
        """Calcula matriz de modularidad"""
        grados = {nodo: 0.0 for nodo in self.nodos}
        peso_total = 0.0

        for u, v, data in self.grafo.edges(data=True):
            peso = abs(data.get('weight', 1.0))
            peso_total += peso
            grados[u] += peso
            grados[v] += peso

        if peso_total == 0:
            return None, 0.0

        B = {}
        for u, v, data in self.grafo.edges(data=True):
            peso = abs(data.get('weight', 1.0))
            i, j = self.nodo_a_idx[u], self.nodo_a_idx[v]
            valor_b = peso - (grados[u] * grados[v]) / (2.0 * peso_total)
            B[(i, j)] = valor_b
            B[(j, i)] = valor_b

        return B, peso_total

    def _obtener_modularidad(self, B, i, j=None):
        if j is None:
            return B.get((i, i), 0.0)
        return B.get((i, j), 0.0) + B.get((j, i), 0.0)

    def _calcular_Q_total(self, B, activos):
        Q = 0.0
        for i in activos:
            for j in activos:
                if i != j:
                    Q += self._obtener_modularidad(B, i, j)
                else:
                    Q += B.get((i, i), 0.0)
        return Q / 2.0

    def ejecutar(self):
        """Ejecuta el algoritmo y retorna comunidades, Q_max, historial"""

        if self.n == 0:
            return [], 0.0, []

        B, peso_total = self._inicializar_modularidad()

        if B is None or not B:
            return [[nodo] for nodo in self.nodos], 0.0, []

        # Comunidades iniciales
        comunidades = {i: {self.idx_a_nodo[i]} for i in range(self.n)}
        a = [B.get((i, i), 0.0) for i in range(self.n)]
        activos = set(range(self.n))

        Q_inicial = self._calcular_Q_total(B, activos)
        Q_mejor = Q_inicial
        partition_mejor = [comunidades[i].copy() for i in activos]

        historial = []
        paso = 0

        while len(activos) > 1:
            paso += 1
            mejor_delta_Q = float('-inf')
            mejor_par = None

            activos_lista = sorted(list(activos))

            for idx_i in range(len(activos_lista)):
                for idx_j in range(idx_i + 1, len(activos_lista)):
                    i, j = activos_lista[idx_i], activos_lista[idx_j]
                    delta_Q = 2.0 * (self._obtener_modularidad(B, i, j) - a[i] * a[j])

                    if delta_Q > mejor_delta_Q:
                        mejor_delta_Q = delta_Q
                        mejor_par = (i, j)

            if mejor_par is None:
                break

            i, j = mejor_par
            Q_actual = self._calcular_Q_total(B, activos)
            Q_nuevo = Q_actual + mejor_delta_Q

            # Información de comunidades
            comm_i = sorted(list(comunidades[i]))
            comm_j = sorted(list(comunidades[j]))
            comm_i_str = "{" + ", ".join(comm_i) + "}"
            comm_j_str = "{" + ", ".join(comm_j) + "}"

            historial.append({
                'Paso': paso,
                'Comunidad_A': comm_i_str,
                'Comunidad_B': comm_j_str,
                'Delta_Q': mejor_delta_Q,
                'Q_anterior': Q_actual,
                'Q_nuevo': Q_nuevo,
                'Num_Comunidades': len(activos) - 1
            })

            # Fusionar
            comunidades[i] = comunidades[i].union(comunidades[j])
            del comunidades[j]

            # Actualizar B
            for col in range(self.n):
                B[(i, col)] = B.get((i, col), 0.0) + B.get((j, col), 0.0)
                B[(col, i)] = B.get((col, i), 0.0) + B.get((col, j), 0.0)

            nuevo_diagonal = B.get((i, i), 0.0) + B.get((j, j), 0.0) + B.get((j, i), 0.0)
            B[(i, i)] = nuevo_diagonal

            for col in range(self.n):
                B.pop((j, col), None)
                B.pop((col, j), None)

            a[i] = a[i] + a[j]
            a[j] = 0.0
            activos.discard(j)

            if Q_nuevo > Q_mejor:
                Q_mejor = Q_nuevo
                partition_mejor = [comunidades[k].copy() for k in activos]

        return partition_mejor, Q_mejor, historial


# ========================================
# FUNCIÓN PRINCIPAL - IMPRIME RESULTADOS
# ========================================

def ejecutar_newman_y_mostrar(df_particion, nombre_particion, threshold=0.01):
    """
    Ejecuta Newman en una partición e IMPRIME todos los resultados
    """

    print(f"\n{'='*130}")
    print(f"ALGORITMO DE NEWMAN - {nombre_particion.upper()}")
    print(f"{'='*130}")

    # Crear grafo desde correlaciones
    G = nx.Graph()

    # Agregar nodos (todas las columnas numéricas)
    cols_numericas = []
    for col in df_particion.columns:
        try:
            float(df_particion[col].iloc[0])
            cols_numericas.append(col)
        except:
            pass

    G.add_nodes_from(cols_numericas)

    # Calcular correlaciones y agregar aristas
    def calcular_correlacion(col1, col2):
        n = len(col1)
        media_x = sum(col1) / n
        media_y = sum(col2) / n
        suma_xy = sum((col1[i] - media_x) * (col2[i] - media_y) for i in range(n))
        suma_x2 = sum((col1[i] - media_x) ** 2 for i in range(n))
        suma_y2 = sum((col2[i] - media_y) ** 2 for i in range(n))
        if suma_x2 == 0 or suma_y2 == 0:
            return 0.0
        return suma_xy / (suma_x2 * suma_y2) ** 0.5

    aristas_agregadas = 0
    for i in range(len(cols_numericas)):
        for j in range(i + 1, len(cols_numericas)):
            col1_vals = [float(df_particion[cols_numericas[i]].iloc[k]) for k in range(len(df_particion))]
            col2_vals = [float(df_particion[cols_numericas[j]].iloc[k]) for k in range(len(df_particion))]
            corr = abs(calcular_correlacion(col1_vals, col2_vals))

            if corr >= threshold:
                G.add_edge(cols_numericas[i], cols_numericas[j], weight=corr)
                aristas_agregadas += 1

    print(f"\n📊 INFORMACIÓN DEL GRAFO:")
    print(f"   • Nodos:       {G.number_of_nodes()}")
    print(f"   • Aristas:     {G.number_of_edges()}")
    print(f"   • Threshold:   {threshold}")

    if G.number_of_nodes() == 0:
        print(f"\n   ⚠️  Grafo vacío. Sin análisis posible.")
        return None

    # Ejecutar Newman
    print(f"\n{'─'*130}")
    print(f"EJECUTANDO ALGORITMO DE NEWMAN...")
    print(f"{'─'*130}\n")

    algoritmo = NewmanAlgorithm(G, nombre_particion=nombre_particion)
    comunidades, Q_max, historial = algoritmo.ejecutar()

    # IMPRIMIR RESULTADOS INICIALES
    print(f"PASO 0: ESTADO INICIAL")
    print(f"   • Modularidad inicial (Q):  {0.0:.8f}")
    print(f"   • Comunidades:              {G.number_of_nodes()}")
    print(f"   • Configuración:            Cada nodo es su propia comunidad\n")

    # IMPRIMIR CADA PASO
    for paso_info in historial:
        print(f"PASO {paso_info['Paso']}: FUSIÓN")
        print(f"   • Fusionando: {paso_info['Comunidad_A']} + {paso_info['Comunidad_B']}")
        print(f"   • ΔQ:         {paso_info['Delta_Q']:+.8f}")
        print(f"   • Q anterior: {paso_info['Q_anterior']:.8f}")
        print(f"   • Q nuevo:    {paso_info['Q_nuevo']:.8f}")
        print(f"   • Comunidades activas: {paso_info['Num_Comunidades']}\n")

    # RESULTADOS FINALES
    print(f"{'='*130}")
    print(f"✅ RESULTADOS FINALES")
    print(f"{'='*130}")
    print(f"   • Q Máximo:                 {Q_max:.8f}")
    print(f"   • Comunidades detectadas:   {len(comunidades)}")
    print(f"   • Pasos ejecutados:         {len(historial)}")
    print(f"\n📋 COMUNIDADES DETECTADAS:")

    for idx, comunidad in enumerate(comunidades, 1):
        nodos_sorted = sorted(list(comunidad))
        print(f"   [{idx}] {nodos_sorted} ({len(nodos_sorted)} variable(s))")

    print(f"\n{'='*130}\n")

    return {
        'comunidades': comunidades,
        'Q_max': Q_max,
        'historial': historial,
        'num_comunidades': len(comunidades)
    }


# ========================================
# EJECUTAR EN TODAS LAS PARTICIONES
# ========================================

def ejecutar_newman_todas_particiones(particiones_lista):
    """
    Ejecuta Newman en todas las particiones y IMPRIME resultados

    particiones_lista: lista de tuplas [(nombre, dataframe), ...]
    """

    print("\n\n" + "🔷"*65)
    print("APLICANDO ALGORITMO DE NEWMAN A TODAS LAS PARTICIONES")
    print("🔷"*65)

    resultados_todos = {}

    for nombre, df_part in particiones_lista:
        resultado = ejecutar_newman_y_mostrar(df_part, nombre, threshold=0.01)
        if resultado:
            resultados_todos[nombre] = resultado

    # TABLA COMPARATIVA FINAL
    print("\n" + "="*130)
    print("TABLA COMPARATIVA: RESULTADOS POR PARTICIÓN")
    print("="*130)
    print(f"{'Partición':<15} {'Nodos':<10} {'Aristas':<10} {'Comunidades':<15} {'Q_máximo':<18} {'Pasos':<10}")
    print("─"*130)

    for nombre, resultado in resultados_todos.items():
        print(f"{nombre:<15} {resultado.get('nodos', '?'):<10} {resultado.get('aristas', '?'):<10} "
              f"{resultado['num_comunidades']:<15} {resultado['Q_max']:<18.8f} "
              f"{len(resultado['historial']):<10}")

    print("="*130)

    # ANÁLISIS: MEJOR Y PEOR Q
    print("\n" + "="*130)
    print("ANÁLISIS: MEJOR Y PEOR MODULARIDAD")
    print("="*130)

    if resultados_todos:
        mejor_nombre = max(resultados_todos, key=lambda x: resultados_todos[x]['Q_max'])
        peor_nombre = min(resultados_todos, key=lambda x: resultados_todos[x]['Q_max'])

        mejor = resultados_todos[mejor_nombre]
        peor = resultados_todos[peor_nombre]

        print(f"\n🥇 MEJOR Q: {mejor_nombre.upper()}")
        print(f"   • Q_máximo:         {mejor['Q_max']:.8f}")
        print(f"   • Comunidades:      {mejor['num_comunidades']}")
        print(f"   • Pasos:            {len(mejor['historial'])}")

        print(f"\n🥈 PEOR Q: {peor_nombre.upper()}")
        print(f"   • Q_máximo:         {peor['Q_max']:.8f}")
        print(f"   • Comunidades:      {peor['num_comunidades']}")
        print(f"   • Pasos:            {len(peor['historial'])}")

        print(f"\n📊 DIFERENCIA: {mejor['Q_max'] - peor['Q_max']:+.8f}")

    print("="*130 + "\n")

    return resultados_todos


In [ ]:
particiones_para_newman = [
    ('df_original', df),
    ('B2C', B2C),
    ('W2C', W2C),
    ('B4C', B4C),
    ('W4C', W4C),
    ('B8C', B8C),
    ('W8C', W8C),
    ('B16C', B16C),
    ('W16C', W16C)
]

resultados_newman = ejecutar_newman_todas_particiones(particiones_para_newman)

In [ ]:
# =================================================================
# PASO 6: GENERANDO ÁRBOLES ENRAIZADOS Y PODADOS POR NIVEL
# =================================================================

print("\n" + "="*90)
print("PASO 6: GENERANDO ÁRBOLES ENRAIZADOS EN 'y' (PODADOS POR NIVEL)")
print("="*90)

# ⭐ PARÁMETRO DE UMBRAL DE NIVEL ⭐
# Nivel 0 = 'y'
# Nivel 1 = Hijos directos de 'y'
# Nivel 2 = Hijos de los hijos
LEVEL_THRESHOLD = 2

print(f"✓ Umbral de Nivel (Profundidad) fijado en: {LEVEL_THRESHOLD}\n")

# Verificar disponibilidad de graphviz
layout_jerarquico_disponible = False
try:
    from networkx.drawing import nx_pydot
    layout_jerarquico_disponible = True
    print("✓ Se encontró 'nx_pydot'. Usando layout jerárquico 'dot'.\n")
except ImportError:
    print("⚠️  'nx_pydot' no disponible. Usando Shell Layout.\n")

# -----------------------------------------------------------------
# FUNCIONES DE RECORRIDO (Sin cambios)
# -----------------------------------------------------------------

def custom_bfs(graph, start_node):
    """Recorrido BFS desde nodo inicial."""
    if start_node not in graph.nodes():
        return []

    visited = set()
    queue = deque([start_node])
    bfs_order = []

    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            bfs_order.append(node)
            # Asegurarse de que el grafo tenga la función neighbors
            if hasattr(graph, 'neighbors'):
                for neighbor in graph.neighbors(node):
                    if neighbor not in visited:
                        queue.append(neighbor)

    return bfs_order

def custom_dfs(graph, start_node):
    """Recorrido DFS desde nodo inicial."""
    if start_node not in graph.nodes():
        return []

    visited = set()
    stack = [start_node]
    dfs_order = []

    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            dfs_order.append(node)
            if hasattr(graph, 'neighbors'):
                neighbors = list(graph.neighbors(node))
                for neighbor in reversed(neighbors):
                    if neighbor not in visited:
                        stack.append(neighbor)

    return dfs_order

# Diccionario para almacenar resultados
resultados_arboles = {}

def crear_arbol_enraizado_particion(partition_name, mst_prim_graph, numero_paso):
    """
    Crea árbol enraizado, lo PODA según el LEVEL_THRESHOLD,
    y luego ejecuta el análisis (layout, viz, BFS/DFS) sobre el árbol podado.
    """

    print(f"\n{'='*90}")
    print(f"PASO {numero_paso}: PARTICIÓN '{partition_name.upper()}'")
    print(f"{'='*90}\n")

    # ==========================================
    # 1. VALIDAR QUE EXISTA NODO 'y'
    # ==========================================
    root_node = 'y'
    if root_node not in mst_prim_graph.nodes():
        print(f"  ✗ Nodo raíz '{root_node}' no encontrado. Saltando.")
        return None

    print(f"1. Creando árbol enraizado en '{root_node}'...")

    # ==========================================
    # 2. CREAR ÁRBOL ENRAIZADO (BFS TREE)
    # ==========================================
    # Primero creamos el árbol completo para saber todas las profundidades
    tree_prim_y = nx.bfs_tree(mst_prim_graph, source=root_node)
    print(f"  ✓ Árbol BFS completo creado ({tree_prim_y.number_of_nodes()} nodos, {tree_prim_y.number_of_edges()} aristas)")

    # ==========================================
    # 2.5 APLICAR PODA POR NIVEL (THRESHOLD)
    # ==========================================
    print(f"\n2. Aplicando poda (Nivel <= {LEVEL_THRESHOLD})...")

    # 1. Encontrar todos los nodos dentro del umbral
    nodes_to_keep = []
    # Calculamos profundidades desde el árbol completo
    heights_full = nx.shortest_path_length(tree_prim_y, source=root_node)

    for node, depth in heights_full.items():
        if depth <= LEVEL_THRESHOLD:
            nodes_to_keep.append(node)

    # 2. Crear el subgrafo basado en esos nodos
    # Usamos .subgraph() y lo copiamos para tener un grafo independiente
    pruned_tree = tree_prim_y.subgraph(nodes_to_keep).copy()

    print(f"  ✓ Árbol podado: {pruned_tree.number_of_nodes()} nodos, {pruned_tree.number_of_edges()} aristas")


    # ==========================================
    # 3. CALCULAR LAYOUT JERÁRQUICO (SOBRE ÁRBOL PODADO)
    # ==========================================
    print(f"\n3. Calculando layout jerárquico...")

    pos = None
    layout_name = "Shell (Fallback)"

    # Calcular layout usando el PRUNED_TREE
    if layout_jerarquico_disponible:
        try:
            pos = nx_pydot.graphviz_layout(pruned_tree, prog='dot') # <--- USA ÁRBOL PODADO
            layout_name = "Jerárquico (dot)"
            print(f"  ✓ Layout 'dot' aplicado")
        except Exception as e:
            print(f"  ⚠️  Layout 'dot' falló. Usando Shell Layout.")
            depths = nx.shortest_path_length(pruned_tree, source=root_node) # <--- USA ÁRBOL PODADO
            levels = {}
            for node, depth in depths.items():
                if depth not in levels:
                    levels[depth] = []
                levels[depth].append(node)
            shells = [levels[d] for d in sorted(levels.keys())]
            pos = nx.shell_layout(pruned_tree, nlist=shells) # <--- USA ÁRBOL PODADO
            layout_name = "Shell (Fallback)"
    else:
        print(f"  ⚠️  nx_pydot no disponible. Usando Shell Layout.")
        depths = nx.shortest_path_length(pruned_tree, source=root_node) # <--- USA ÁRBOL PODADO
        levels = {}
        for node, depth in depths.items():
            if depth not in levels:
                levels[depth] = []
            levels[depth].append(node)
        shells = [levels[d] for d in sorted(levels.keys())]
        pos = nx.shell_layout(pruned_tree, nlist=shells) # <--- USA ÁRBOL PODADO
        layout_name = "Shell (Fallback)"

    # ==========================================
    # 4. VISUALIZAR ÁRBOL PODADO
    # ==========================================
    print(f"\n4. Generando visualización (podada)...")

    fig, ax = plt.subplots(1, 1, figsize=(18, 14))
    # Título actualizado para reflejar la poda
    fig.suptitle(f"ÁRBOL PODADO (Nivel <= {LEVEL_THRESHOLD}) EN '{root_node}' - MST PRIM\nPartición: {partition_name.upper()}",
                 fontsize=16, fontweight='bold')

    # Colorear nodo raíz
    node_colors = ['#ff0000' if node == root_node else '#90EE90' for node in pruned_tree.nodes()] # <--- USA ÁRBOL PODADO

    # Dibujar árbol dirigido (podado)
    nx.draw(
        pruned_tree, # <--- USA ÁRBOL PODADO
        pos,
        ax=ax,
        with_labels=True,
        node_size=3500,
        node_color=node_colors,
        edge_color='#333333',
        width=2.5,
        font_size=12,
        font_weight='bold',
        arrows=True,
        arrowsize=30,
        arrowstyle='-|>',
        edgecolors='black',
        linewidths=2.5,
        connectionstyle='arc3,rad=0.1'
    )

    ax.set_title(f"Layout: {layout_name} | Raíz: '{root_node}'",
                fontsize=13, fontweight='bold', pad=15)
    ax.axis('off')

    plt.tight_layout()

    # Guardar figura
    filename = f"paso_{numero_paso:02d}_{partition_name}_arbol_enraizado_PODADO.png" # <--- Nombre actualizado
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"  ✓ Visualización guardada: '{filename}'")
    plt.show()

    # ==========================================
    # 5. EJECUTAR BFS Y DFS (SOBRE ÁRBOL PODADO)
    # ==========================================
    print(f"\n5. Ejecutando recorridos BFS y DFS (sobre árbol podado)...")

    bfs_result = custom_bfs(pruned_tree, root_node) # <--- USA ÁRBOL PODADO
    dfs_result = custom_dfs(pruned_tree, root_node) # <--- USA ÁRBOL PODADO

    print(f"\n  BFS (Amplitud):")
    print(f"      {' → '.join(bfs_result)}")
    print(f"      Nodos visitados: {len(bfs_result)}")

    print(f"\n  DFS (Profundidad):")
    print(f"      {' → '.join(dfs_result)}")
    print(f"      Nodos visitados: {len(dfs_result)}")

    # ==========================================
    # 6. ESTADÍSTICAS DEL ÁRBOL (PODADO)
    # ==========================================
    heights = nx.shortest_path_length(pruned_tree, source=root_node) # <--- USA ÁRBOL PODADO
    altura = max(heights.values()) if heights else 0
    profundidad_promedio = sum(heights.values()) / len(heights) if heights else 0

    print(f"\n6. Estadísticas del árbol (podado):") # <--- Título actualizado
    print(f"  • Altura (profundidad máxima): {altura}")
    print(f"  • Profundidad promedio: {profundidad_promedio:.2f}")
    # Usar pruned_tree para el cálculo
    branch_factor = (pruned_tree.number_of_edges() / max(1, pruned_tree.number_of_nodes() - 1)) if pruned_tree.number_of_nodes() > 1 else 0
    print(f"  • Factor de ramificación: {branch_factor:.2f}")

    # ==========================================
    # 7. GUARDAR RESULTADOS (DEL ÁRBOL PODADO)
    # ==========================================
    resultados_arboles[partition_name] = {
        'arbol_enraizado': pruned_tree, # <--- Guarda el árbol podado
        'posiciones': pos,
        'bfs': bfs_result,
        'dfs': dfs_result,
        'nodos_totales': pruned_tree.number_of_nodes(), # <--- Stats del podado
        'aristas_totales': pruned_tree.number_of_edges(), # <--- Stats del podado
        'altura': altura,
        'profundidad_promedio': profundidad_promedio,
        'layout': layout_name
    }

    print(f"\n{'─'*90}")
    print(f"✓ Partición '{partition_name.upper()}' procesada (podada)")
    print(f"{'─'*90}\n")

    return resultados_arboles[partition_name]

# ========================================
# EJECUTAR PARA TODAS LAS PARTICIONES
# (Esta sección no necesita cambios,
#  simplemente llamará a la función modificada)
# ========================================

print("\nObteniendo MST Prim de resultados previos...\n")

# Usar los MST Prim ya calculados
if 'resultados_pearson' not in globals() or 'GraphAlgorithms' not in globals():
    print("⚠️  ERROR: 'resultados_pearson' o 'GraphAlgorithms' no se encontró.")
    print("Asegúrate de ejecutar primero el análisis de correlación y visualización de grafos (script anterior).")
else:
    particiones_lista = [
        ('df_original', df),
        ('B2C', B2C), ('W2C', W2C),
        ('B4C', B4C), ('W4C', W4C),
        ('B8C', B8C), ('W8C', W8C),
        ('B16C', B16C), ('W16C', W16C)
    ]

    paso = 0
    for nombre, partition_df in particiones_lista:
        paso += 1
        try:
            # Obtener columnas numéricas
            numerical_cols = []
            if not partition_df.empty:
                for col in partition_df.columns:
                    # Usar pd.to_numeric para ser más robusto
                    if pd.to_numeric(partition_df[col], errors='coerce').notna().any():
                         numerical_cols.append(col)

            # Obtener aristas del resultado previo (si existe)
            if nombre in resultados_pearson:
                aristas = resultados_pearson[nombre]['aristas']

                # Recrear grafo
                G = nx.Graph()
                G.add_nodes_from(numerical_cols)
                if aristas:
                    for arista in aristas:
                        r = arista['r_pearson']
                        weight = 1.0 - r
                        G.add_edge(arista['var1'], arista['var2'], weight=weight, correlation=r)

                # Calcular MST Prim
                if G.number_of_edges() > 0:
                    # Usar nuestra implementación de Prim (del script anterior)
                    mst_prim, _ = GraphAlgorithms.prim_mst(G)
                    # Llamar a la función (modificada)
                    crear_arbol_enraizado_particion(nombre, mst_prim, paso)
                else:
                    print(f"\n{'='*90}")
                    print(f"PASO {paso}: PARTICIÓN '{nombre.upper()}'")
                    print(f"{'='*90}\n")
                    print(f"⚠️  Sin aristas en {nombre}. Saltando.\n")
            else:
                print(f"\n{'='*90}")
                print(f"PASO {paso}: PARTICIÓN '{nombre.upper()}'")
                print(f"{'='*90}\n")
                print(f"⚠️  {nombre} no encontrado en resultados_pearson. Saltando.\n")

        except Exception as e:
            print(f"\n⚠️  Error en '{nombre}': {str(e)}")
            traceback.print_exc()
            print(f"  Continuando...\n")

# ========================================
# RESUMEN COMPARATIVO
# ========================================

if resultados_arboles:
    print("\n" + "="*90)
    print(f"RESUMEN COMPARATIVO DE ÁRBOLES ENRAIZADOS (PODADOS A NIVEL <= {LEVEL_THRESHOLD})")
    print("="*90)

    for partition_name, data in resultados_arboles.items():
        print(f"\n{partition_name.upper()}:")
        print(f"  Nodos: {data['nodos_totales']} | Aristas: {data['aristas_totales']} | Altura: {data['altura']}")
        print(f"  BFS: {' → '.join(data['bfs'])}")
        print(f"  DFS: {' → '.join(data['dfs'])}")

    # ========================================
    # TABLA COMPARATIVA
    # ========================================

    print("\n" + "="*90)
    print(f"TABLA COMPARATIVA DE ÁRBOLES ENRAIZADOS (PODADOS A NIVEL <= {LEVEL_THRESHOLD})")
    print("="*90 + "\n")

    tabla_arboles = []
    for partition_name, data in resultados_arboles.items():
        tabla_arboles.append({
            'Partición': partition_name,
            'Nodos': data['nodos_totales'],
            'Aristas': data['aristas_totales'],
            'Altura': data['altura'],
            'Prof_Promedio': f"{data['profundidad_promedio']:.2f}",
            'Layout': data['layout']
        })

    df_arboles = pd.DataFrame(tabla_arboles)
    print(df_arboles.to_string(index=False))

    # Guardar tabla
    df_arboles.to_csv('resumen_arboles_enraizados_podados.csv', index=False)
    print("\n✓ Tabla guardada: 'resumen_arboles_enraizados_podados.csv'")

    print("\n" + "="*90)
    print("✓ GENERACIÓN DE ÁRBOLES PODADOS COMPLETADA")
    print("="*90)
else:
    print("\nNo se generaron resultados de árboles enraizados.")

In [ ]:
print("\n" + "="*100)
print("VISUALIZACIÓN: GRAFO ORIGINAL + MST KRUSKAL + MST PRIM CON CAMINOS MÁS LARGOS RESALTADOS")
print("="*100)

# Verificar disponibilidad de pydot/graphviz
layout_jerarquico_disponible = False
try:
    from networkx.drawing import nx_pydot
    layout_jerarquico_disponible = True
    print("✓ Se encontró 'nx_pydot'. Usando layout jerárquico 'dot'.\n")
except ImportError:
    print("⚠️  'nx_pydot' no disponible. Usando layout de resorte.\n")

# =================================================================
# FUNCIÓN AUXILIAR PARA ENCONTRAR EL CAMINO MÁS LARGO (DIÁMETRO)
# =================================================================

def encontrar_camino_mas_largo(T):
    """
    Encuentra el camino más largo (diámetro) entre dos hojas en un árbol.
    Devuelve el camino (lista de nodos) y su longitud (número de aristas).
    """
    if T.number_of_edges() == 0:
        return [], 0

    # Encontrar todas las hojas (nodos con grado 1)
    leaves = [node for node in T.nodes() if T.degree(node) == 1]

    # Caso especial: un grafo de 2 nodos y 1 arista (ambos son hojas)
    if len(leaves) < 2 and T.number_of_nodes() == 2:
        leaves = list(T.nodes())
    elif len(leaves) < 2:
        return [], 0

    longest_path = []
    max_len = -1

    # Encontrar el camino más largo entre todos los pares de hojas
    for i in range(len(leaves)):
        for j in range(i + 1, len(leaves)):
            try:
                path = nx.shortest_path(T, source=leaves[i], target=leaves[j])
                path_len = len(path) - 1  # Longitud en aristas

                if path_len > max_len:
                    max_len = path_len
                    longest_path = path
            except nx.NetworkXNoPath:
                continue

    return longest_path, max_len


# ========================================
# FUNCIÓN PRINCIPAL DE VISUALIZACIÓN
# ========================================

def visualizar_grafos_mst_particion(partition_name, aristas_filtradas, numerical_cols, numero_paso, threshold):
    """
    Visualiza para una partición:
    1. Grafo original de correlaciones
    2. MST Kruskal con camino resaltado
    3. MST Prim con camino resaltado
    """

    print(f"\n{'='*100}")
    print(f"PASO {numero_paso}: PARTICIÓN '{partition_name.upper()}' (threshold={threshold})")
    print(f"{'='*100}\n")

    # ==========================================
    # 1. CREAR GRAFO ORIGINAL
    # ==========================================
    print(f"1. Creando grafo de correlaciones...")

    G_temp = nx.Graph()
    G_temp.add_nodes_from(numerical_cols)

    if not aristas_filtradas:
        print(f"   ⚠️  Sin aristas significativas. Saltando.")
        return

    for arista in aristas_filtradas:
        r = arista['r_pearson']
        weight = 1.0 - r  # Para MST (busca peso mínimo)
        G_temp.add_edge(arista['var1'], arista['var2'], weight=weight, correlation=r)

    print(f"   ✓ Grafo: {G_temp.number_of_nodes()} nodos, {G_temp.number_of_edges()} aristas")

    # ==========================================
    # 2. CALCULAR MST KRUSKAL Y PRIM
    # ==========================================
    print(f"\n2. Calculando MST Kruskal y Prim...")

    if G_temp.number_of_edges() == 0:
        print(f"   ✗ Sin aristas. Saltando.")
        return

    # MST Kruskal
    mst_kruskal_temp = nx.minimum_spanning_tree(G_temp, weight='weight', algorithm='kruskal')
    peso_kruskal = sum(data['weight'] for u, v, data in mst_kruskal_temp.edges(data=True))

    # MST Prim
    mst_prim_temp = nx.minimum_spanning_tree(G_temp, weight='weight', algorithm='prim')
    peso_prim = sum(data['weight'] for u, v, data in mst_prim_temp.edges(data=True))

    print(f"   ✓ Kruskal: {mst_kruskal_temp.number_of_nodes()} nodos, {mst_kruskal_temp.number_of_edges()} aristas, Peso: {peso_kruskal:.4f}")
    print(f"   ✓ Prim: {mst_prim_temp.number_of_nodes()} nodos, {mst_prim_temp.number_of_edges()} aristas, Peso: {peso_prim:.4f}")

    # ==========================================
    # 3. ENCONTRAR CAMINOS MÁS LARGOS
    # ==========================================
    print(f"\n3. Encontrando caminos más largos (diámetro)...")

    path_k, len_k = encontrar_camino_mas_largo(mst_kruskal_temp)
    print(f"   ✓ Kruskal - Camino más largo ({len_k} aristas): {' → '.join(path_k)}")

    path_p, len_p = encontrar_camino_mas_largo(mst_prim_temp)
    print(f"   ✓ Prim - Camino más largo ({len_p} aristas): {' → '.join(path_p)}")

    # ==========================================
    # 4. CALCULAR LAYOUTS
    # ==========================================
    print(f"\n4. Calculando layouts...")

    pos_original = None
    pos_kruskal = None
    pos_prim = None
    layout_name = "Spring"

    if layout_jerarquico_disponible:
        try:
            pos_original = nx_pydot.graphviz_layout(G_temp, prog='neato')
            pos_kruskal = nx_pydot.graphviz_layout(mst_kruskal_temp, prog='dot')
            pos_prim = nx_pydot.graphviz_layout(mst_prim_temp, prog='dot')
            layout_name = "Jerárquico (dot)"
            print(f"   ✓ Layout jerárquico aplicado")
        except Exception as e:
            print(f"   ⚠️  Layout 'dot' falló. Usando Spring Layout.")
            pos_original = nx.spring_layout(G_temp, seed=42, k=2, iterations=200)
            pos_kruskal = nx.spring_layout(mst_kruskal_temp, seed=42, k=2, iterations=200)
            pos_prim = nx.spring_layout(mst_prim_temp, seed=42, k=2, iterations=200)
            layout_name = "Spring (Fallback)"
    else:
        print(f"   ⚠️  nx_pydot no disponible. Usando Spring Layout.")
        pos_original = nx.spring_layout(G_temp, seed=42, k=2, iterations=200)
        pos_kruskal = nx.spring_layout(mst_kruskal_temp, seed=42, k=2, iterations=200)
        pos_prim = nx.spring_layout(mst_prim_temp, seed=42, k=2, iterations=200)
        layout_name = "Spring (Fallback)"

    # ==========================================
    # 5. CREAR VISUALIZACIÓN 1x3
    # ==========================================
    print(f"\n5. Generando visualización...")

    fig, axes = plt.subplots(1, 3, figsize=(26, 10))
    fig.suptitle(f"PARTICIÓN: {partition_name.upper()} | Filas: {len(aristas_filtradas)} | Threshold: {threshold} | Layout: {layout_name}",
                 fontsize=17, fontweight='bold', y=0.98)

    # --- GRAFO 1: ORIGINAL ---
    ax1 = axes[0]

    if G_temp.number_of_edges() > 0:
        edge_widths_original = [abs(G_temp[u][v].get('weight', 1)) * 6 for u, v in G_temp.edges()]
        edge_colors_original = ['#0066cc' if G_temp[u][v].get('correlation', 1) > 0 else '#ff3333'
                                for u, v in G_temp.edges()]

        nx.draw_networkx_edges(G_temp, pos_original, ax=ax1, width=edge_widths_original,
                               edge_color=edge_colors_original, alpha=0.6)

    nx.draw_networkx_nodes(G_temp, pos_original, ax=ax1, node_size=2500, node_color='lightblue',
                           edgecolors='navy', linewidths=2.5)
    nx.draw_networkx_labels(G_temp, pos_original, ax=ax1, font_size=11, font_weight='bold')

    ax1.set_title(f"GRAFO ORIGINAL\nNodos: {G_temp.number_of_nodes()} | Aristas: {G_temp.number_of_edges()}",
                  fontsize=13, fontweight='bold', pad=10)
    ax1.axis('off')

    # --- GRAFO 2: MST KRUSKAL CON CAMINO RESALTADO ---
    ax2 = axes[1]

    path_nodes_k = set(path_k)
    other_nodes_k = set(mst_kruskal_temp.nodes()) - path_nodes_k

    path_edges_k = list(zip(path_k[:-1], path_k[1:]))
    other_edges_k = set(mst_kruskal_temp.edges())
    for u, v in path_edges_k:
        other_edges_k.discard((u, v))
        other_edges_k.discard((v, u))

    # Dibujar aristas normales
    if other_edges_k:
        nx.draw_networkx_edges(mst_kruskal_temp, pos_kruskal, ax=ax2, edgelist=other_edges_k,
                               width=3, edge_color='#00aa00', alpha=0.6)

    # Dibujar nodos normales
    if other_nodes_k:
        nx.draw_networkx_nodes(mst_kruskal_temp, pos_kruskal, ax=ax2, nodelist=other_nodes_k,
                               node_size=2500, node_color='lightgreen',
                               edgecolors='darkgreen', linewidths=2.5)

    # Dibujar aristas del camino
    if path_edges_k:
        nx.draw_networkx_edges(mst_kruskal_temp, pos_kruskal, ax=ax2, edgelist=path_edges_k,
                               width=7, edge_color='red', alpha=1.0, style='solid')

    # Dibujar nodos del camino
    if path_nodes_k:
        nx.draw_networkx_nodes(mst_kruskal_temp, pos_kruskal, ax=ax2, nodelist=path_nodes_k,
                               node_size=2800, node_color='gold',
                               edgecolors='black', linewidths=3)

    nx.draw_networkx_labels(mst_kruskal_temp, pos_kruskal, ax=ax2, font_size=11, font_weight='bold')

    ax2.set_title(f"MST KRUSKAL (Camino máximo: {len_k})\nNodos: {mst_kruskal_temp.number_of_nodes()} | Aristas: {mst_kruskal_temp.number_of_edges()}",
                  fontsize=13, fontweight='bold', pad=10)
    ax2.axis('off')

    # --- GRAFO 3: MST PRIM CON CAMINO RESALTADO ---
    ax3 = axes[2]

    path_nodes_p = set(path_p)
    other_nodes_p = set(mst_prim_temp.nodes()) - path_nodes_p

    path_edges_p = list(zip(path_p[:-1], path_p[1:]))
    other_edges_p = set(mst_prim_temp.edges())
    for u, v in path_edges_p:
        other_edges_p.discard((u, v))
        other_edges_p.discard((v, u))

    # Dibujar aristas normales
    if other_edges_p:
        nx.draw_networkx_edges(mst_prim_temp, pos_prim, ax=ax3, edgelist=other_edges_p,
                               width=3, edge_color='#cc0066', alpha=0.6)

    # Dibujar nodos normales
    if other_nodes_p:
        nx.draw_networkx_nodes(mst_prim_temp, pos_prim, ax=ax3, nodelist=other_nodes_p,
                               node_size=2500, node_color='lightcoral',
                               edgecolors='darkred', linewidths=2.5)

    # Dibujar aristas del camino
    if path_edges_p:
        nx.draw_networkx_edges(mst_prim_temp, pos_prim, ax=ax3, edgelist=path_edges_p,
                               width=7, edge_color='red', alpha=1.0, style='solid')

    # Dibujar nodos del camino
    if path_nodes_p:
        nx.draw_networkx_nodes(mst_prim_temp, pos_prim, ax=ax3, nodelist=path_nodes_p,
                               node_size=2800, node_color='gold',
                               edgecolors='black', linewidths=3)

    nx.draw_networkx_labels(mst_prim_temp, pos_prim, ax=ax3, font_size=11, font_weight='bold')

    ax3.set_title(f"MST PRIM (Camino máximo: {len_p})\nNodos: {mst_prim_temp.number_of_nodes()} | Aristas: {mst_prim_temp.number_of_edges()}",
                  fontsize=13, fontweight='bold', pad=10)
    ax3.axis('off')

    plt.tight_layout()

    filename = f"paso_{numero_paso:02d}_{partition_name}_grafos_mst_camino_resaltado.png"
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"   ✓ Visualización guardada: '{filename}'")
    plt.show()

    # ==========================================
    # 6. ESTADÍSTICAS
    # ==========================================
    print(f"\n6. Estadísticas:")
    print(f"   {'─'*90}")

    degree_seq = [G_temp.degree(n) for n in G_temp.nodes()]
    print(f"   GRAFO ORIGINAL:")
    print(f"       • Densidad: {nx.density(G_temp):.4f}")
    print(f"       • Grado Promedio: {sum(degree_seq)/len(degree_seq) if degree_seq else 0:.2f}")
    print(f"       • Grado Máximo: {max(degree_seq) if degree_seq else 0}")

    print(f"\n   MST KRUSKAL:")
    print(f"       • Peso Total: {peso_kruskal:.4f}")
    print(f"       • Camino más largo: {len_k} aristas")

    print(f"\n   MST PRIM:")
    print(f"       • Peso Total: {peso_prim:.4f}")
    print(f"       • Camino más largo: {len_p} aristas")

    print(f"   {'─'*90}\n")

    return {
        'kruskal_camino_len': len_k,
        'prim_camino_len': len_p,
        'peso_kruskal': peso_kruskal,
        'peso_prim': peso_prim
    }


# ========================================
# EJECUTAR VISUALIZACIÓN
# ========================================

print("\nObtienendo datos del análisis previo...\n")

if 'resultados_pearson' not in globals():
    print("⚠️  ERROR: 'resultados_pearson' no se encontró.")
    print("Asegúrate de ejecutar primero el análisis de correlación y visualización de grafos.")
else:
    resultados_visualizacion = {}
    paso = 0

    for partition_name, data in resultados_pearson.items():
        paso += 1

        try:
            aristas = data.get('aristas', [])
            # Usar el threshold guardado en resultados_pearson, o usar el principal si no existe
            threshold = data.get('threshold', THRESHOLD if 'THRESHOLD' in globals() else 0.75)

            # Obtener columnas numéricas
            if partition_name in particiones_analizar:
                partition_df = particiones_analizar[partition_name]
                numerical_cols = []
                for col in partition_df.columns:
                    try:
                        _ = float(partition_df[col].iloc[0])
                        numerical_cols.append(col)
                    except:
                        pass

                # Visualizar
                resultado = visualizar_grafos_mst_particion(partition_name, aristas, numerical_cols, paso, threshold)
                if resultado:
                    resultados_visualizacion[partition_name] = resultado

        except Exception as e:
            print(f"\n⚠️  Error en '{partition_name}': {str(e)}")
            traceback.print_exc()
            print(f"   Continuando...\n")

    # ========================================
    # TABLA RESUMIDA
    # ========================================

    if resultados_visualizacion:
        print("\n" + "="*100)
        print("TABLA RESUMIDA: CAMINOS MÁS LARGOS Y PESOS MST")
        print("="*100 + "\n")

        tabla_resumen = []
        for partition_name, resultado in resultados_visualizacion.items():
            tabla_resumen.append({
                'Partición': partition_name,
                'Camino_Kruskal': resultado['kruskal_camino_len'],
                'Camino_Prim': resultado['prim_camino_len'],
                'Peso_Kruskal': f"{resultado['peso_kruskal']:.4f}",
                'Peso_Prim': f"{resultado['peso_prim']:.4f}"
            })

        df_resumen = pd.DataFrame(tabla_resumen)
        print(df_resumen.to_string(index=False))

        df_resumen.to_csv('resumen_caminos_mst.csv', index=False)
        print("\n✓ Tabla guardada: 'resumen_caminos_mst.csv'")

print("\n" + "="*100)
print("✓ VISUALIZACIÓN COMPLETADA")
print("="*100)

In [ ]:
print("\n" + "="*100)
print("VISUALIZACIÓN: MST PRIM (COMPLETO + CAMINO) vs MST PRIM (PODADO)")
print("="*100)

# Verificar disponibilidad de pydot/graphviz
layout_jerarquico_disponible = False
try:
    from networkx.drawing import nx_pydot
    layout_jerarquico_disponible = True
    print("✓ Se encontró 'nx_pydot'. Usando layout jerárquico 'dot'.\n")
except ImportError:
    print("⚠️  'nx_pydot' no disponible. Usando layout de resorte.\n")

# =================================================================
# FUNCIÓN AUXILIAR PARA ENCONTRAR EL CAMINO MÁS LARGO (DIÁMETRO)
# =================================================================

def encontrar_camino_mas_largo(T):
    """
    Encuentra el camino más largo (diámetro) entre dos hojas en un árbol.
    Devuelve el camino (lista de nodos) y su longitud (número de aristas).
    """
    if T.number_of_edges() == 0:
        return [], 0

    leaves = [node for node in T.nodes() if T.degree(node) == 1]

    if len(leaves) < 2 and T.number_of_nodes() == 2:
        leaves = list(T.nodes())
    elif len(leaves) < 2:
        return [], 0

    longest_path = []
    max_len = -1

    for i in range(len(leaves)):
        for j in range(i + 1, len(leaves)):
            try:
                path = nx.shortest_path(T, source=leaves[i], target=leaves[j])
                path_len = len(path) - 1

                if path_len > max_len:
                    max_len = path_len
                    longest_path = path
            except nx.NetworkXNoPath:
                continue

    return longest_path, max_len


# =================================================================
# FUNCIÓN AUXILIAR PARA PODAR EL ÁRBOL
# =================================================================

def podar_arbol_por_la_mitad(T_original, nodo_raiz='y'):
    """
    Encuentra el camino más largo (diámetro), corta el árbol por la mitad
    de ese camino, y devuelve el sub-árbol que contiene el 'nodo_raiz'.
    """
    T = T_original.copy()

    if T.number_of_nodes() < 2 or T.number_of_edges() == 0:
        return T

    if nodo_raiz not in T.nodes():
        print(f"   ⚠️  Advertencia (Poda): Nodo raíz '{nodo_raiz}' no encontrado. Devolviendo MST completo.")
        return T

    # 1. Encontrar el camino más largo
    longest_path, path_len = encontrar_camino_mas_largo(T)

    if not longest_path:
        print(f"   ⚠️  Advertencia (Poda): No se encontró ningún camino. Devolviendo MST completo.")
        return T

    if path_len <= 1:
        print(f"   ℹ️  Info (Poda): Camino más largo ({path_len}) demasiado corto para cortar.")
        try:
            componente_y = nx.node_connected_component(T, nodo_raiz)
            return T.subgraph(componente_y).copy()
        except Exception:
            G_y = nx.Graph()
            G_y.add_node(nodo_raiz)
            return G_y

    # 2. Encontrar la arista a cortar (Lógica Par/Impar)
    target_edge_count = (path_len + (path_len % 2)) // 2

    node_index_a = target_edge_count - 1
    node_index_b = target_edge_count

    u, v = longest_path[node_index_a], longest_path[node_index_b]

    # 3. Cortar el árbol
    if T.has_edge(u, v):
        print(f"   ✂️  Poda: Camino más largo encontrado ({path_len} aristas).")
        print(f"   ✂️  Poda: Cortando arista #{target_edge_count}: ({u}, {v})")
        T.remove_edge(u, v)
    else:
        print(f"   ⚠️  Advertencia (Poda): La arista a cortar ({u}, {v}) no existe. Devolviendo MST completo.")
        return T

    # 4. Quedarse con el sub-árbol que contiene 'y'
    try:
        for component in nx.connected_components(T):
            if nodo_raiz in component:
                print(f"   ✓ Poda: Conservando componente con '{nodo_raiz}' ({len(component)} nodos).")
                return T.subgraph(component).copy()

        print(f"   ⚠️  Advertencia (Poda): Nodo '{nodo_raiz}' quedó aislado. Devolviendo solo el nodo.")
        G_y = nx.Graph()
        G_y.add_node(nodo_raiz)
        return G_y

    except Exception as e:
        print(f"   ⚠️  Error durante la selección de componente: {e}. Devolviendo MST completo.")
        return T_original


# ========================================
# FUNCIÓN PRINCIPAL DE VISUALIZACIÓN
# ========================================

def visualizar_prim_podado_particion(partition_name, aristas_filtradas, numerical_cols, numero_paso, threshold):
    """
    Visualiza para una partición:
    1. Grafo original
    2. MST Prim (completo con camino resaltado)
    3. MST Prim (podado)
    """

    print(f"\n{'='*100}")
    print(f"PASO {numero_paso}: PARTICIÓN '{partition_name.upper()}' (threshold={threshold})")
    print(f"{'='*100}\n")

    # ==========================================
    # 1. CREAR GRAFO ORIGINAL
    # ==========================================
    print(f"1. Creando grafo de correlaciones...")

    G_temp = nx.Graph()
    G_temp.add_nodes_from(numerical_cols)

    if not aristas_filtradas:
        print(f"   ⚠️  Sin aristas significativas. Saltando.")
        return

    for arista in aristas_filtradas:
        r = arista['r_pearson']
        weight = 1.0 - r  # Para MST (busca peso mínimo)
        G_temp.add_edge(arista['var1'], arista['var2'], weight=weight, correlation=r)

    print(f"   ✓ Grafo: {G_temp.number_of_nodes()} nodos, {G_temp.number_of_edges()} aristas")

    # ==========================================
    # 2. CALCULAR MST PRIM
    # ==========================================
    print(f"\n2. Calculando MST Prim...")

    if G_temp.number_of_edges() == 0:
        print(f"   ✗ Sin aristas. Saltando.")
        return

    mst_prim_temp = nx.minimum_spanning_tree(G_temp, weight='weight', algorithm='prim')
    peso_prim = sum(data['weight'] for u, v, data in mst_prim_temp.edges(data=True))

    print(f"   ✓ Prim: {mst_prim_temp.number_of_nodes()} nodos, {mst_prim_temp.number_of_edges()} aristas, Peso: {peso_prim:.4f}")

    # ==========================================
    # 3. ENCONTRAR CAMINO MÁS LARGO
    # ==========================================
    print(f"\n3. Encontrando camino más largo...")

    path_p, len_p = encontrar_camino_mas_largo(mst_prim_temp)
    print(f"   ✓ Camino más largo ({len_p} aristas): {' → '.join(path_p)}")

    # ==========================================
    # 4. PODAR EL ÁRBOL
    # ==========================================
    print(f"\n4. Podando MST Prim...")

    mst_prim_podado = podar_arbol_por_la_mitad(mst_prim_temp, nodo_raiz='y')

    # ==========================================
    # 5. CALCULAR LAYOUTS
    # ==========================================
    print(f"\n5. Calculando layouts...")

    pos_original = None
    pos_prim = None
    pos_prim_podado = None
    layout_name = "Spring"

    if layout_jerarquico_disponible:
        try:
            pos_original = nx_pydot.graphviz_layout(G_temp, prog='neato')
            pos_prim = nx_pydot.graphviz_layout(mst_prim_temp, prog='dot')
            pos_prim_podado = nx_pydot.graphviz_layout(mst_prim_podado, prog='dot')
            layout_name = "Jerárquico (dot)"
            print(f"   ✓ Layout jerárquico aplicado")
        except Exception as e:
            print(f"   ⚠️  Layout 'dot' falló. Usando Spring Layout.")
            pos_original = nx.spring_layout(G_temp, seed=42, k=2, iterations=200)
            pos_prim = nx.spring_layout(mst_prim_temp, seed=42, k=2, iterations=200)
            pos_prim_podado = nx.spring_layout(mst_prim_podado, seed=42, k=2, iterations=200)
            layout_name = "Spring (Fallback)"
    else:
        print(f"   ⚠️  nx_pydot no disponible. Usando Spring Layout.")
        pos_original = nx.spring_layout(G_temp, seed=42, k=2, iterations=200)
        pos_prim = nx.spring_layout(mst_prim_temp, seed=42, k=2, iterations=200)
        pos_prim_podado = nx.spring_layout(mst_prim_podado, seed=42, k=2, iterations=200)
        layout_name = "Spring (Fallback)"

    # ==========================================
    # 6. CREAR VISUALIZACIÓN 1x3
    # ==========================================
    print(f"\n6. Generando visualización...")

    fig, axes = plt.subplots(1, 3, figsize=(26, 10))
    fig.suptitle(f"PARTICIÓN: {partition_name.upper()} | Threshold: {threshold} | Layout: {layout_name}",
                 fontsize=17, fontweight='bold', y=0.98)

    # --- GRAFO 1: ORIGINAL ---
    ax1 = axes[0]

    if G_temp.number_of_edges() > 0:
        edge_widths_original = [abs(G_temp[u][v].get('weight', 1)) * 6 for u, v in G_temp.edges()]
        edge_colors_original = ['#0066cc' if G_temp[u][v].get('correlation', 1) > 0 else '#ff3333'
                                for u, v in G_temp.edges()]
        nx.draw_networkx_edges(G_temp, pos_original, ax=ax1, width=edge_widths_original,
                               edge_color=edge_colors_original, alpha=0.6)

    nx.draw_networkx_nodes(G_temp, pos_original, ax=ax1, node_size=2500, node_color='lightblue',
                           edgecolors='navy', linewidths=2.5)
    nx.draw_networkx_labels(G_temp, pos_original, ax=ax1, font_size=11, font_weight='bold')

    ax1.set_title(f"GRAFO ORIGINAL\nNodos: {G_temp.number_of_nodes()} | Aristas: {G_temp.number_of_edges()}",
                  fontsize=13, fontweight='bold', pad=10)
    ax1.axis('off')

    # --- GRAFO 2: MST PRIM (COMPLETO CON CAMINO RESALTADO) ---
    ax2 = axes[1]

    path_nodes_p = set(path_p)
    other_nodes_p = set(mst_prim_temp.nodes()) - path_nodes_p

    path_edges_p = list(zip(path_p[:-1], path_p[1:]))
    other_edges_p = set(mst_prim_temp.edges())
    for u, v in path_edges_p:
        other_edges_p.discard((u, v))
        other_edges_p.discard((v, u))

    # Dibujar aristas normales
    if other_edges_p:
        nx.draw_networkx_edges(mst_prim_temp, pos_prim, ax=ax2, edgelist=other_edges_p,
                               width=3, edge_color='#cc0066', alpha=0.6)

    # Dibujar nodos normales
    if other_nodes_p:
        nx.draw_networkx_nodes(mst_prim_temp, pos_prim, ax=ax2, nodelist=other_nodes_p,
                               node_size=2500, node_color='lightcoral',
                               edgecolors='darkred', linewidths=2.5)

    # Dibujar aristas del camino
    if path_edges_p:
        nx.draw_networkx_edges(mst_prim_temp, pos_prim, ax=ax2, edgelist=path_edges_p,
                               width=7, edge_color='red', alpha=1.0, style='solid')

    # Dibujar nodos del camino
    if path_nodes_p:
        nx.draw_networkx_nodes(mst_prim_temp, pos_prim, ax=ax2, nodelist=path_nodes_p,
                               node_size=2800, node_color='gold',
                               edgecolors='black', linewidths=3)

    nx.draw_networkx_labels(mst_prim_temp, pos_prim, ax=ax2, font_size=11, font_weight='bold')

    ax2.set_title(f"MST PRIM (Camino máximo: {len_p})\nNodos: {mst_prim_temp.number_of_nodes()} | Aristas: {mst_prim_temp.number_of_edges()}",
                  fontsize=13, fontweight='bold', pad=10)
    ax2.axis('off')

    # --- GRAFO 3: MST PRIM (PODADO) ---
    ax3 = axes[2]

    if mst_prim_podado.number_of_edges() > 0:
        edge_widths_podado = [abs(mst_prim_podado[u][v].get('weight', 1)) * 6 for u, v in mst_prim_podado.edges()]
        edge_colors_podado = ['#00aa00' if mst_prim_podado[u][v].get('correlation', 1) > 0 else '#ff6600'
                               for u, v in mst_prim_podado.edges()]
        nx.draw_networkx_edges(mst_prim_podado, pos_prim_podado, ax=ax3, width=edge_widths_podado,
                               edge_color=edge_colors_podado, alpha=0.85, style='solid')

    nx.draw_networkx_nodes(mst_prim_podado, pos_prim_podado, ax=ax3, node_size=2500, node_color='lightgreen',
                           edgecolors='darkgreen', linewidths=2.5)
    nx.draw_networkx_labels(mst_prim_podado, pos_prim_podado, ax=ax3, font_size=11, font_weight='bold')

    ax3.set_title(f"MST PRIM (PODADO)\nNodos: {mst_prim_podado.number_of_nodes()} | Aristas: {mst_prim_podado.number_of_edges()}",
                  fontsize=13, fontweight='bold', pad=10)
    ax3.axis('off')

    plt.tight_layout()

    filename = f"paso_{numero_paso:02d}_{partition_name}_prim_poda_comparacion.png"
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"   ✓ Visualización guardada: '{filename}'")
    plt.show()

    # ==========================================
    # 7. ESTADÍSTICAS
    # ==========================================
    print(f"\n7. Estadísticas:")
    print(f"   {'─'*90}")

    degree_seq = [G_temp.degree(n) for n in G_temp.nodes()]
    print(f"   GRAFO ORIGINAL:")
    print(f"       • Densidad: {nx.density(G_temp):.4f}")
    print(f"       • Grado Promedio: {sum(degree_seq)/len(degree_seq) if degree_seq else 0:.2f}")

    print(f"\n   MST PRIM (Original):")
    print(f"       • Peso Total: {peso_prim:.4f}")
    print(f"       • Camino más largo: {len_p} aristas")

    print(f"\n   MST PRIM (Podado):")
    print(f"       • Nodos restantes: {mst_prim_podado.number_of_nodes()}")
    print(f"       • Aristas restantes: {mst_prim_podado.number_of_edges()}")

    print(f"   {'─'*90}\n")

    return {
        'camino_len': len_p,
        'peso_prim': peso_prim,
        'nodos_podado': mst_prim_podado.number_of_nodes(),
        'aristas_podado': mst_prim_podado.number_of_edges()
    }


# ========================================
# EJECUTAR VISUALIZACIÓN
# ========================================

print("\nObtienendo datos del análisis previo...\n")

if 'resultados_pearson' not in globals():
    print("⚠️  ERROR: 'resultados_pearson' no se encontró.")
    print("Asegúrate de ejecutar primero el análisis de correlación.")
else:
    resultados_poda = {}
    paso = 0

    for partition_name, data in resultados_pearson.items():
        paso += 1

        try:
            aristas = data.get('aristas', [])
            threshold = data.get('threshold', THRESHOLD if 'THRESHOLD' in globals() else 0.75)

            # Obtener columnas numéricas
            if partition_name in particiones_analizar:
                partition_df = particiones_analizar[partition_name]
                numerical_cols = []
                for col in partition_df.columns:
                    try:
                        _ = float(partition_df[col].iloc[0])
                        numerical_cols.append(col)
                    except:
                        pass

                # Visualizar
                resultado = visualizar_prim_podado_particion(partition_name, aristas, numerical_cols, paso, threshold)
                if resultado:
                    resultados_poda[partition_name] = resultado

        except Exception as e:
            print(f"\n⚠️  Error en '{partition_name}': {str(e)}")
            traceback.print_exc()
            print(f"   Continuando...\n")

    # ========================================
    # TABLA RESUMIDA
    # ========================================

    if resultados_poda:
        print("\n" + "="*100)
        print("TABLA RESUMIDA: COMPARACIÓN PRIM ORIGINAL vs PODADO")
        print("="*100 + "\n")

        tabla_resumen = []
        for partition_name, resultado in resultados_poda.items():
            tabla_resumen.append({
                'Partición': partition_name,
                'Camino_Largo': resultado['camino_len'],
                'Peso_Prim': f"{resultado['peso_prim']:.4f}",
                'Nodos_Podado': resultado['nodos_podado'],
                'Aristas_Podado': resultado['aristas_podado']
            })

        df_resumen = pd.DataFrame(tabla_resumen)
        print(df_resumen.to_string(index=False))

        df_resumen.to_csv('resumen_poda_prim.csv', index=False)
        print("\n✓ Tabla guardada: 'resumen_poda_prim.csv'")

print("\n" + "="*100)
print("✓ VISUALIZACIÓN PODA COMPLETADA")

In [ ]:
# =================================================================
# ETAPA 4: COMPARACIÓN GRÁFICA DE ÁRBOLES PODADOS (DIFERENCIAS)
# INTEGRADA CON ETAPA 3 - Reutiliza árboles ya calculados
# =================================================================
print("\n" + "="*80)
print("ETAPA 4: IDENTIFICACIÓN DE DIFERENCIAS EN ÁRBOLES PODADOS")
print("="*80 + "\n")

# -----------------------------------------------------------------
# VERIFICAR DISPONIBILIDAD DE DATOS DE ETAPA 3
# -----------------------------------------------------------------

if 'mst_prim_trees' not in globals() or 'pruned_mst_trees' not in globals():
    print("⚠️  ADVERTENCIA: Datos de ETAPA 3 no encontrados.")
    print("   Se generarán los árboles podados localmente.")
    print("   (Asegúrate de ejecutar ETAPA 3 primero para máxima eficiencia)\n")

    # Si no existen, crearemos los árboles aquí (fallback)
    mst_prim_trees = {}
    pruned_mst_trees = {}
    USAR_DATOS_ETAPA3 = False
else:
    print("✓ Se encontraron datos de ETAPA 3.")
    print(f"✓ Árboles MST disponibles: {list(mst_prim_trees.keys())}")
    print(f"✓ Árboles podados disponibles: {list(pruned_mst_trees.keys())}\n")
    USAR_DATOS_ETAPA3 = True

# -----------------------------------------------------------------
# FUNCIONES AUXILIARES
# -----------------------------------------------------------------

def encontrar_camino_mas_largo(T):
    """Encuentra el camino más largo (diámetro) entre dos hojas."""
    if T.number_of_edges() == 0:
        return [], 0

    leaves = [node for node in T.nodes() if T.degree(node) == 1]
    if len(leaves) < 2:
        if T.number_of_nodes() == 1:
            return list(T.nodes()), 0
        elif T.number_of_nodes() == 2:
            return list(T.nodes()), 1
        else:
            return [], 0

    longest_path = []
    max_len = -1

    for i in range(len(leaves)):
        for j in range(i + 1, len(leaves)):
            try:
                path = nx.shortest_path(T, source=leaves[i], target=leaves[j])
                path_len = len(path) - 1
                if path_len > max_len:
                    max_len = path_len
                    longest_path = path
            except nx.NetworkXNoPath:
                continue

    return longest_path, max_len


def podar_arbol_por_la_mitad(T_original, nodo_raiz_fijo='y'):
    """Corta el árbol por la mitad y devuelve sub-árbol con nodo_raiz_fijo."""
    T = T_original.copy()

    if T.number_of_nodes() < 2 or T.number_of_edges() == 0:
        return T

    if nodo_raiz_fijo not in T.nodes():
        if T.number_of_nodes() > 0:
            return T.subgraph([list(T.nodes())[0]]).copy()
        return T

    longest_path, path_len = encontrar_camino_mas_largo(T)

    if not longest_path or path_len <= 1:
        try:
            component = nx.node_connected_component(T, nodo_raiz_fijo)
            return T.subgraph(component).copy()
        except Exception:
            G_y = nx.Graph()
            G_y.add_node(nodo_raiz_fijo)
            return G_y

    target_edge_count = (path_len + (path_len % 2)) // 2
    node_index_a = target_edge_count - 1
    node_index_b = target_edge_count

    if node_index_b >= len(longest_path):
        return T_original.copy()

    u, v = longest_path[node_index_a], longest_path[node_index_b]

    if T.has_edge(u, v):
        T.remove_edge(u, v)

    try:
        for component in nx.connected_components(T):
            if nodo_raiz_fijo in component:
                return T.subgraph(component).copy()
        G_y = nx.Graph()
        G_y.add_node(nodo_raiz_fijo)
        return G_y
    except Exception:
        return T_original.copy()


def obtener_estructura_arbol(T, root):
    """
    Devuelve un diccionario {nodo: nivel_desde_raiz} para cada nodo del árbol.
    """
    estructura = {}
    if T.number_of_nodes() == 0:
        return estructura

    try:
        for nodo in T.nodes():
            try:
                nivel = nx.shortest_path_length(T, root, nodo)
                estructura[nodo] = nivel
            except nx.NetworkXNoPath:
                estructura[nodo] = None
    except Exception as e:
        print(f"   ⚠️  Error obtener estructura: {e}")

    return estructura


def comparar_estructuras(struct_b, struct_w, nombre_b, nombre_w):
    """
    Compara las estructuras de dos árboles y retorna SOLO las diferencias.
    Retorna un DataFrame con nodos que NO están en la misma posición.
    """
    diferencias = []

    # Nodos comunes
    nodos_comunes = set(struct_b.keys()) & set(struct_w.keys())

    # Nodos solo en B
    solo_en_b = set(struct_b.keys()) - set(struct_w.keys())

    # Nodos solo en W
    solo_en_w = set(struct_w.keys()) - set(struct_b.keys())

    # 1. Diferencias en NIVEL (nodos que están en ambos pero con diferente profundidad)
    for nodo in nodos_comunes:
        nivel_b = struct_b[nodo]
        nivel_w = struct_w[nodo]

        if nivel_b != nivel_w:  # Solo si están en diferente nivel
            diferencias.append({
                'Tipo': 'Diferente Nivel',
                'Nodo': nodo,
                f'Nivel en {nombre_b}': nivel_b,
                f'Nivel en {nombre_w}': nivel_w,
                'Diferencia': abs(nivel_b - nivel_w) if (nivel_b is not None and nivel_w is not None) else 'N/A'
            })

    # 2. Nodos que existen solo en B
    for nodo in solo_en_b:
        diferencias.append({
            'Tipo': f'Solo en {nombre_b}',
            'Nodo': nodo,
            f'Nivel en {nombre_b}': struct_b[nodo],
            f'Nivel en {nombre_w}': '-',
            'Diferencia': 'Ausente'
        })

    # 3. Nodos que existen solo en W
    for nodo in solo_en_w:
        diferencias.append({
            'Tipo': f'Solo en {nombre_w}',
            'Nodo': nodo,
            f'Nivel en {nombre_b}': '-',
            f'Nivel en {nombre_w}': struct_w[nodo],
            'Diferencia': 'Ausente'
        })

    if not diferencias:
        return None

    df = pd.DataFrame(diferencias)
    return df.sort_values('Nodo').reset_index(drop=True)


def visualizar_comparacion(T_b, T_w, root_b, root_w, nombre_b, nombre_w, df_diferencias):
    """
    Dibuja los dos árboles resaltando SOLO los nodos con diferencias.
    """
    print(f"\n   🎨 Generando visualización comparativa...")

    nodos_b = set(T_b.nodes())
    nodos_w = set(T_w.nodes())

    # Obtener nodos con diferencias
    if df_diferencias is not None:
        nodos_diferentes = set(df_diferencias['Nodo'].unique())
    else:
        nodos_diferentes = set()

    comunes = nodos_b.intersection(nodos_w) - nodos_diferentes  # Comunes SIN diferencias
    solo_en_b = nodos_b - nodos_w
    solo_en_w = nodos_w - nodos_b

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))
    fig.suptitle(f"Comparación: {nombre_b} vs {nombre_w} | Diferencias Destacadas",
                 fontsize=16, fontweight='bold')

    # Layouts
    pos_b = nx.spring_layout(T_b, seed=42, k=2, iterations=50) if T_b.number_of_nodes() > 0 else {}
    pos_w = nx.spring_layout(T_w, seed=42, k=2, iterations=50) if T_w.number_of_nodes() > 0 else {}

    # --- Árbol B ---
    ax1.set_title(f"{nombre_b} ({len(nodos_b)} nodos)", fontsize=14, fontweight='bold')
    if pos_b:
        # Nodos comunes (sin diferencias) - gris claro
        if comunes:
            nx.draw_networkx_nodes(T_b, pos_b, ax=ax1, nodelist=comunes, node_size=2000,
                                  node_color='lightgray', edgecolors='black', linewidths=1, label='Igual posición')

        # Nodos con diferencias - rojo
        nodos_diff_b = nodos_diferentes & nodos_b
        if nodos_diff_b:
            nx.draw_networkx_nodes(T_b, pos_b, ax=ax1, nodelist=nodos_diff_b, node_size=2800,
                                  node_color='#FF6B6B', edgecolors='darkred', linewidths=3, label='Diferencia detectada')

        # Nodos solo en B - amarillo
        if solo_en_b:
            nx.draw_networkx_nodes(T_b, pos_b, ax=ax1, nodelist=solo_en_b, node_size=2800,
                                  node_color='#FFD93D', edgecolors='#FF9F1C', linewidths=3, label=f'Solo en {nombre_b}')

        # Raíz - oro
        if root_b and root_b in T_b.nodes():
            nx.draw_networkx_nodes(T_b, pos_b, ax=ax1, nodelist=[root_b], node_size=3200,
                                  node_color='gold', edgecolors='black', linewidths=3, label='Raíz')

        nx.draw_networkx_edges(T_b, pos_b, ax=ax1, edge_color='gray', width=2, alpha=0.6)
        nx.draw_networkx_labels(T_b, pos_b, ax=ax1, font_size=9, font_weight='bold')
        ax1.legend(loc='upper left', fontsize=10)
    ax1.axis('off')

    # --- Árbol W ---
    ax2.set_title(f"{nombre_w} ({len(nodos_w)} nodos)", fontsize=14, fontweight='bold')
    if pos_w:
        # Nodos comunes (sin diferencias) - gris claro
        if comunes:
            nx.draw_networkx_nodes(T_w, pos_w, ax=ax2, nodelist=comunes, node_size=2000,
                                  node_color='lightgray', edgecolors='black', linewidths=1, label='Igual posición')

        # Nodos con diferencias - rojo
        nodos_diff_w = nodos_diferentes & nodos_w
        if nodos_diff_w:
            nx.draw_networkx_nodes(T_w, pos_w, ax=ax2, nodelist=nodos_diff_w, node_size=2800,
                                  node_color='#FF6B6B', edgecolors='darkred', linewidths=3, label='Diferencia detectada')

        # Nodos solo en W - azul
        if solo_en_w:
            nx.draw_networkx_nodes(T_w, pos_w, ax=ax2, nodelist=solo_en_w, node_size=2800,
                                  node_color='#4ECDC4', edgecolors='#1A7F7E', linewidths=3, label=f'Solo en {nombre_w}')

        # Raíz - oro
        if root_w and root_w in T_w.nodes():
            nx.draw_networkx_nodes(T_w, pos_w, ax=ax2, nodelist=[root_w], node_size=3200,
                                  node_color='gold', edgecolors='black', linewidths=3, label='Raíz')

        nx.draw_networkx_edges(T_w, pos_w, ax=ax2, edge_color='gray', width=2, alpha=0.6)
        nx.draw_networkx_labels(T_w, pos_w, ax=ax2, font_size=9, font_weight='bold')
        ax2.legend(loc='upper left', fontsize=10)
    ax2.axis('off')

    plt.tight_layout()
    filename = f"comparacion_{nombre_b}_vs_{nombre_w}_diferencias.png"
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"   ✓ Visualización guardada: '{filename}'")
    plt.show()


# -----------------------------------------------------------------
# PASO 1: PREPARAR ÁRBOLES PODADOS
# -----------------------------------------------------------------

print("Preparando árboles podados para comparación...\n")

pares_a_comparar = [('B2C', 'W2C'), ('B4C', 'W4C'), ('B8C', 'W8C'), ('B16C', 'W16C')]
todas_las_diferencias = {}

if not USAR_DATOS_ETAPA3:
    print("⚠️  Generando árboles localmente (fallback)...\n")

    if 'particiones_analizar' not in globals():
        print("❌ ERROR: 'particiones_analizar' no disponible. Abortando.")
    else:
        # Generar MSTs para pares faltantes
        for nombre_b, nombre_w in pares_a_comparar:
            if nombre_b not in pruned_mst_trees and nombre_b in particiones_analizar:
                print(f"   Generando árbol podado para {nombre_b}...")
                df_temp = particiones_analizar[nombre_b]
                corr_matrix = df_temp.corr()
                G_temp = nx.Graph()
                for i in range(len(corr_matrix.columns)):
                    for j in range(i+1, len(corr_matrix.columns)):
                        if abs(corr_matrix.iloc[i, j]) > 0.01:
                            G_temp.add_edge(corr_matrix.columns[i], corr_matrix.columns[j],
                                          weight=1.0 - abs(corr_matrix.iloc[i, j]))

                if G_temp.number_of_edges() > 0:
                    mst_temp = nx.minimum_spanning_tree(G_temp, weight='weight')
                    pruned_mst_trees[nombre_b] = podar_arbol_por_la_mitad(mst_temp, nodo_raiz_fijo='y')

            if nombre_w not in pruned_mst_trees and nombre_w in particiones_analizar:
                print(f"   Generando árbol podado para {nombre_w}...")
                df_temp = particiones_analizar[nombre_w]
                corr_matrix = df_temp.corr()
                G_temp = nx.Graph()
                for i in range(len(corr_matrix.columns)):
                    for j in range(i+1, len(corr_matrix.columns)):
                        if abs(corr_matrix.iloc[i, j]) > 0.01:
                            G_temp.add_edge(corr_matrix.columns[i], corr_matrix.columns[j],
                                          weight=1.0 - abs(corr_matrix.iloc[i, j]))

                if G_temp.number_of_edges() > 0:
                    mst_temp = nx.minimum_spanning_tree(G_temp, weight='weight')
                    pruned_mst_trees[nombre_w] = podar_arbol_por_la_mitad(mst_temp, nodo_raiz_fijo='y')

# -----------------------------------------------------------------
# PASO 2: EJECUTAR COMPARACIÓN POR PARES
# -----------------------------------------------------------------

print("\n" + "="*80)
print("COMPARANDO PARES DE ÁRBOLES PODADOS")
print("="*80 + "\n")

for nombre_b, nombre_w in pares_a_comparar:
    print(f"\n{'='*80}")
    print(f"COMPARANDO: {nombre_b} vs {nombre_w}")
    print(f"{'='*80}\n")

    try:
        # 1. Obtener árboles podados
        if nombre_b not in pruned_mst_trees or nombre_w not in pruned_mst_trees:
            print(f"   ⚠️  Árboles no disponibles para esta comparación. Saltando...")
            continue

        T_b = pruned_mst_trees[nombre_b]
        T_w = pruned_mst_trees[nombre_w]

        print(f"   ✓ Árboles cargados desde ETAPA 3")

        # 2. Estadísticas
        print(f"\n   {nombre_b}: {T_b.number_of_nodes()} nodos, {T_b.number_of_edges()} aristas")
        print(f"   {nombre_w}: {T_w.number_of_nodes()} nodos, {T_w.number_of_edges()} aristas")

        # 3. Encontrar raíces (centro del árbol)
        root_b = nx.center(T_b)[0] if T_b.number_of_nodes() > 0 else None
        root_w = nx.center(T_w)[0] if T_w.number_of_nodes() > 0 else None
        print(f"\n   Raíz {nombre_b}: '{root_b}'")
        print(f"   Raíz {nombre_w}: '{root_w}'")

        # 4. Obtener estructuras de ambos árboles
        print(f"\n   📊 Analizando estructuras...")
        struct_b = obtener_estructura_arbol(T_b, root_b)
        struct_w = obtener_estructura_arbol(T_w, root_w)

        # 5. Comparar y obtener SOLO diferencias
        df_diferencias = comparar_estructuras(struct_b, struct_w, nombre_b, nombre_w)

        # 6. Mostrar resultados
        if df_diferencias is None or len(df_diferencias) == 0:
            print(f"\n   ✓ NO HAY DIFERENCIAS - Ambos árboles tienen la misma estructura.")
        else:
            print(f"\n   ⚠️  DIFERENCIAS ENCONTRADAS: {len(df_diferencias)}")
            print("\n" + "="*80)
            print(df_diferencias.to_string(index=False))
            print("="*80)
            todas_las_diferencias[f"{nombre_b} vs {nombre_w}"] = df_diferencias

        # 7. Visualizar con diferencias resaltadas
        visualizar_comparacion(T_b, T_w, root_b, root_w, nombre_b, nombre_w, df_diferencias)

    except Exception as e:
        print(f"   ❌ ERROR: {e}")
        traceback.print_exc()

# -----------------------------------------------------------------
# RESUMEN FINAL
# -----------------------------------------------------------------

print("\n\n" + "="*80)
print("📋 RESUMEN DE DIFERENCIAS")
print("="*80)

if todas_las_diferencias:
    for comparacion, df_diff in todas_las_diferencias.items():
        print(f"\n{comparacion}: ({len(df_diff)} diferencias)")
        print("-" * 80)
        print(df_diff.to_string(index=False))
else:
    print("\n✓ SIN DIFERENCIAS EN NINGUNA COMPARACIÓN")

# Guardar resumen
if todas_las_diferencias:
    df_todas = pd.concat([df_diff.assign(Comparacion=comp)
                          for comp, df_diff in todas_las_diferencias.items()],
                         ignore_index=True)
    df_todas.to_csv('resumen_diferencias_arboles.csv', index=False)
    print(f"\n✓ Resumen guardado: 'resumen_diferencias_arboles.csv'")

print("\n" + "="*80)
print("✓ ANÁLISIS DE DIFERENCIAS FINALIZADO")
print("="*80)

In [ ]:
# ========================================
# SCRIPT 8: UNIÓN DE RECORRIDOS POR PARES
#
# 1. Une BFS y DFS para 'B2C'.
# 2. Une BFS y DFS para 'W2C'.
# 3. Une los dos resultados (B2C y W2C)
#    en un solo array final para "2C".
# 4. Repite para 4C, 8C, 16C.
# ========================================

import traceback

def _get_union_for_partition(partition_name, results_dict):
    """
    Función auxiliar para obtener la unión BFS/DFS de UNA partición.
    Imprime los pasos intermedios.
    """
    print(f"\n  Procesando sub-partición: {partition_name}")

    if partition_name not in results_dict:
        print("    ⚠️  Partición no encontrada en 'resultados_arboles'. Saltando.")
        return set() # Devuelve un set vacío

    data = results_dict[partition_name]
    bfs_list = data.get('bfs')
    dfs_list = data.get('dfs')

    if bfs_list is None or dfs_list is None:
        print("    ⚠️  Datos de BFS o DFS no encontrados en esta partición. Saltando.")
        return set() # Devuelve un set vacío

    print(f"    Recorrido BFS ({len(bfs_list)} nodos): {' → '.join(bfs_list)}")
    print(f"    Recorrido DFS ({len(dfs_list)} nodos): {' → '.join(dfs_list)}")

    # Realizar la unión interna (BFS y DFS de esta partición)
    union_set = set(bfs_list).union(set(dfs_list))
    print(f"    Sub-unión ({len(union_set)} nodos): {sorted(list(union_set))}")

    return union_set

def unir_recorridos_por_pares():

    print("\n" + "="*80)
    print("INICIANDO SCRIPT 8: UNIÓN DE RECORRIDOS POR PARES (B vs W)")
    print("="*80)

    # 1. Verificar si 'resultados_arboles' existe
    try:
        if 'resultados_arboles' not in locals() and 'resultados_arboles' not in globals():
             print("\n⚠️ ERROR: La variable 'resultados_arboles' no se encontró.")
             print("Por favor, asegúrate de ejecutar el 'PASO 6' (script de árboles enraizados) primero.")
             return

        global resultados_arboles

        if not resultados_arboles:
             print("\n⚠️ ADVERTENCIA: La variable 'resultados_arboles' está vacía. Nada que procesar.")
             return

        print(f"\nSe encontraron {len(resultados_arboles)} particiones en 'resultados_arboles'.")

        # 2. Definir los pares a procesar
        pares_a_comparar = [
            ('B2C', 'W2C'),
            ('B4C', 'W4C'),
            ('B8C', 'W8C'),
            ('B16C', 'W16C')
        ]

        uniones_finales = {}

        # 3. --- Procesar Pares ---
        for nombre_b, nombre_w in pares_a_comparar:

            # Obtener el nombre del par (ej: "2C")
            pair_name = nombre_b[1:]

            print(f"\n{'='*70}")
            print(f"PROCESANDO PAR: {pair_name} (Combinando '{nombre_b}' y '{nombre_w}')")
            print(f"{'='*70}")

            # 3a. Obtener la unión interna de la partición "Best"
            set_b = _get_union_for_partition(nombre_b, resultados_arboles)

            # 3b. Obtener la unión interna de la partición "Worst"
            set_w = _get_union_for_partition(nombre_w, resultados_arboles)

            # 3c. Calcular la UNIÓN TOTAL entre B y W
            total_union_set = set_b.union(set_w)
            total_union_list = sorted(list(total_union_set))

            uniones_finales[pair_name] = total_union_list

            # 4. Imprimir el resultado final para el par
            print(f"\n  --- UNIÓN TOTAL PARA '{pair_name}' ---")
            print(f"  Nodos Únicos ({len(total_union_list)}):")
            print(f"  {total_union_list}")

        # 5. --- Procesar 'df_original' por separado (no tiene par) ---
        print(f"\n{'='*70}")
        print(f"PROCESANDO: df_original (sin par)")
        print(f"{'='*70}")

        set_orig = _get_union_for_partition('df_original', resultados_arboles)
        uniones_finales['df_original'] = sorted(list(set_orig))
        # La función _get_union_for_partition ya imprime el resultado,
        # así que no necesitamos una sección de "UNIÓN TOTAL".

        print("\n" + "="*80)
        print("PROCESO DE UNIÓN POR PARES COMPLETADO")
        print("="*80)

        # Opcional: puedes acceder a los resultados finales en el dict
        # return uniones_finales

    except NameError:
        print("\n❌ ERROR CRÍTICO: La variable 'resultados_arboles' no está definida.")
        print("Por favor, asegúrate de ejecutar el 'PASO 6' primero.")
    except Exception as e:
        print(f"\n❌ Ocurrió un error inesperado: {e}")
        traceback.print_exc()

# --- Ejecutar la función ---
unir_recorridos_por_pares()

In [ ]:
# ========================================
# SCRIPT 9: EXTRAER NODOS CON DISCREPANCIAS
#
# Lee el diccionario 'todas_las_diferencias'
# (generado por la Etapa 4) y extrae una
# lista (array) de todos los nodos que
# mostraron diferencias.
# ========================================

import traceback
import pandas as pd

def extraer_nodos_discrepantes():

    print("\n" + "="*80)
    print("INICIANDO SCRIPT 9: EXTRACCIÓN DE NODOS CON DISCREPANCIAS")
    print("="*80)

    # 1. Verificar si 'todas_las_diferencias' existe
    try:
        # La variable debe existir globalmente de la celda anterior
        if 'todas_las_diferencias' not in locals() and 'todas_las_diferencias' not in globals():
             print("\n⚠️ ERROR: La variable 'todas_las_diferencias' no se encontró.")
             print("Por favor, asegúrate de ejecutar la 'ETAPA 4' (comparación de árboles) primero.")
             return

        global todas_las_diferencias

        if not todas_las_diferencias:
             print("\n⚠️ ADVERTENCIA: La variable 'todas_las_diferencias' está vacía.")
             print("   (No se encontraron diferencias en la Etapa 4).")
             return

        print(f"\nSe encontraron {len(todas_las_diferencias)} comparaciones en 'todas_las_diferencias'.")

        # Diccionario para guardar los arrays
        nodos_discrepantes_por_par = {}

        # 2. Iterar sobre cada par de comparación
        for comparacion, df_diff in todas_las_diferencias.items():

            print(f"\n{'='*70}")
            print(f"PROCESANDO COMPARACIÓN: {comparacion.upper()}")
            print(f"{'='*70}")

            if not isinstance(df_diff, pd.DataFrame) or 'Nodo' not in df_diff.columns:
                print(f"  ⚠️  Datos para '{comparacion}' no son un DataFrame válido o falta la columna 'Nodo'. Saltando.")
                continue

            # --- Tarea Principal ---
            # 1. Obtener la columna 'Nodo'
            # 2. Usar .unique() para obtener cada nodo una sola vez
            # 3. Convertir a una lista (array)
            nodos_con_discrepancia = df_diff['Nodo'].unique().tolist()

            # 4. Ordenar alfabéticamente para una salida limpia
            nodos_con_discrepancia.sort()

            # 5. Guardar el resultado
            nodos_discrepantes_por_par[comparacion] = nodos_con_discrepancia

            # 6. Imprimir el array
            print(f"  Total de nodos con discrepancias: {len(nodos_con_discrepancia)}")
            print(f"  ARRAY DE NODOS:")
            print(f"  {nodos_con_discrepancia}")

        print("\n" + "="*80)
        print("EXTRACCIÓN DE NODOS COMPLETADA")
        print("="*80)

        # Puedes usar 'nodos_discrepantes_por_par' en celdas futuras
        # return nodos_discrepantes_por_par

    except NameError:
        print("\n❌ ERROR CRÍTICO: La variable 'todas_las_diferencias' no está definida.")
        print("Por favor, asegúrate de ejecutar la 'ETAPA 4' primero.")
    except Exception as e:
        print(f"\n❌ Ocurrió un error inesperado: {e}")
        traceback.print_exc()

# --- Ejecutar la función ---
extraer_nodos_discrepantes()

In [ ]:
# ======================================================
# SCRIPT 10: INTERSECCIÓN DE RESULTADOS DE ANÁLISIS
#
# Compara los resultados de dos metodologías:
# 1. (Recorridos): Nodos cercanos a 'y' (de Script 8)
# 2. (Discrepancias): Nodos estructuralmente diferentes (de Script 9)
#
# Encuentra los nodos que están en AMBAS listas.
# ======================================================

import traceback
import pandas as pd
from collections import deque # Necesario para la lógica de BFS

# --- PASO 1: Lógica de Script 8 (Unión de Recorridos) ---
# Necesitamos la función auxiliar de Script 8
def _get_union_for_partition(partition_name, results_dict):
    """
    Función auxiliar para obtener la unión BFS/DFS de UNA partición.
    (Versión silenciosa, solo devuelve el set)
    """
    if partition_name not in results_dict:
        return set() # Devuelve un set vacío

    data = results_dict[partition_name]
    bfs_list = data.get('bfs')
    dfs_list = data.get('dfs')

    if bfs_list is None or dfs_list is None:
        return set() # Devuelve un set vacío

    union_set = set(bfs_list).union(set(dfs_list))
    return union_set

def generar_uniones_de_recorridos(resultados_arboles):
    """
    Ejecuta la lógica de Script 8 para generar el dict de uniones.
    """
    print("\n--- PASO 1: Generando 'Array 1' (Unión de Recorridos)... ---")

    pares_a_comparar = [
        ('B2C', 'W2C'), ('B4C', 'W4C'),
        ('B8C', 'W8C'), ('B16C', 'W16C')
    ]
    uniones_finales = {}

    # Procesar Pares
    for nombre_b, nombre_w in pares_a_comparar:
        pair_name = nombre_b[1:] # Ej: "2C"
        set_b = _get_union_for_partition(nombre_b, resultados_arboles)
        set_w = _get_union_for_partition(nombre_w, resultados_arboles)

        # Calcular la UNIÓN TOTAL entre B y W
        total_union_set = set_b.union(set_w)
        uniones_finales[pair_name] = total_union_set

    # Procesar 'df_original'
    set_orig = _get_union_for_partition('df_original', resultados_arboles)
    uniones_finales['df_original'] = set_orig

    print("✓ 'Array 1' (Recorridos) generado.")
    return uniones_finales

# --- PASO 2: Lógica de Script 9 (Nodos Discrepantes) ---
def generar_nodos_discrepantes(todas_las_diferencias):
    """
    Ejecuta la lógica de Script 9 para generar el dict de discrepancias.
    """
    print("\n--- PASO 2: Generando 'Array 2' (Nodos con Discrepancias)... ---")

    nodos_discrepantes_por_par = {}

    for comparacion, df_diff in todas_las_diferencias.items():
        if not isinstance(df_diff, pd.DataFrame) or 'Nodo' not in df_diff.columns:
            continue

        # Obtener nodos únicos y guardarlos como un set
        nodos_con_discrepancia_set = set(df_diff['Nodo'].unique())
        nodos_discrepantes_por_par[comparacion] = nodos_con_discrepancia_set

    print("✓ 'Array 2' (Discrepancias) generado.")
    return nodos_discrepantes_por_par

# --- PASO 3: INTERSECCIÓN (¡Nuevo!) ---
def encontrar_interseccion_final(uniones_finales, nodos_discrepantes):
    """
    Compara los dos diccionarios y encuentra la intersección.
    """
    print("\n" + "="*80)
    print("PASO 3: REALIZANDO INTERSECCIÓN DE RESULTADOS")
    print("="*80)

    # Mapa para alinear los keys de los dos diccionarios
    # Llave = (Key de Discrepancias), Valor = (Key de Recorridos)
    key_map = {
        'B2C vs W2C': '2C',
        'B4C vs W4C': '4C',
        'B8C vs W8C': '8C',
        'B16C vs W16C': '16C'
        # df_original no tiene discrepancias, se ignora
    }

    resultados_de_interseccion = {}

    # Iteramos sobre los resultados de las Discrepancias (Array 2)
    for comparacion_key, set_discrepantes in nodos_discrepantes.items():

        print(f"\n{'='*70}")
        print(f"COMPARACIÓN: {comparacion_key.upper()}")
        print(f"{'='*70}")

        # 1. Encontrar la key de Recorridos (Array 1)
        if comparacion_key not in key_map:
            print(f"  - No se encontró un par para '{comparacion_key}' en el mapa. Saltando.")
            continue

        recorrido_key = key_map[comparacion_key] # Ej: '2C'

        # 2. Obtener los dos sets
        set_recorridos = uniones_finales.get(recorrido_key)

        if set_recorridos is None:
            print(f"  - No se encontraron datos de recorrido para '{recorrido_key}'. Saltando.")
            continue

        # 3. Imprimir las dos listas de entrada
        print(f"  Array 1 (Recorridos {recorrido_key}):")
        print(f"    {sorted(list(set_recorridos))}")

        print(f"\n  Array 2 (Discrepancias {comparacion_key}):")
        print(f"    {sorted(list(set_discrepantes))}")

        # 4. Calcular la INTERSECCIÓN
        interseccion = set_recorridos.intersection(set_discrepantes)
        lista_interseccion = sorted(list(interseccion))

        resultados_de_interseccion[comparacion_key] = lista_interseccion

        # 5. Imprimir el resultado final
        print(f"\n  --- INTERSECCIÓN FINAL ({len(lista_interseccion)} nodos) ---")
        print(f"  (Nodos que son CERCANOS a 'y' Y ESTRUCTURALMENTE INESTABLES)")
        print(f"  {lista_interseccion}")

    return resultados_de_interseccion

# --- Función Principal ---
def ejecutar_analisis_de_interseccion():

    print("\n" + "="*80)
    print("INICIANDO SCRIPT 10: ANÁLISIS DE INTERSECCIÓN")
    print("="*80)

    # 1. Verificar dependencias
    try:
        if ('resultados_arboles' not in locals() and 'resultados_arboles' not in globals()) or \
           ('todas_las_diferencias' not in locals() and 'todas_las_diferencias' not in globals()):
             print("\n⚠️ ERROR: Faltan variables de scripts anteriores.")
             print("Por favor, asegúrate de haber ejecutado:")
             print("  1. 'PASO 6' (para generar 'resultados_arboles')")
             print("  2. 'ETAPA 4' (para generar 'todas_las_diferencias')")
             return

        global resultados_arboles, todas_las_diferencias

        # --- PASO 1 ---
        # (Lógica de Script 8)
        uniones_finales = generar_uniones_de_recorridos(resultados_arboles)

        # --- PASO 2 ---
        # (Lógica de Script 9)
        nodos_discrepantes = generar_nodos_discrepantes(todas_las_diferencias)

        # --- PASO 3 ---
        # (Nueva Lógica de Intersección)
        resultados_finales = encontrar_interseccion_final(uniones_finales, nodos_discrepantes)

        print("\n" + "="*80)
        print("SCRIPT 10: ANÁLISIS DE INTERSECCIÓN COMPLETADO")
        print("="*80)

        # 'resultados_finales' está disponible para su uso
        # return resultados_finales

    except NameError as e:
        print(f"\n❌ ERROR CRÍTICO: Faltan variables: {e}")
        print("Por favor, asegúrate de ejecutar las celdas 'PASO 6' y 'ETAPA 4' primero.")
    except Exception as e:
        print(f"\n❌ Ocurrió un error inesperado: {e}")
        traceback.print_exc()

# --- Ejecutar la función ---
ejecutar_analisis_de_interseccion()

In [ ]:
# ========================================
# PASO FINAL: GUARDAR TODOS LOS ARCHIVOS (OPCIONAL)
# ========================================
print("\n" + "="*80)
print("PASO FINAL: GUARDANDO TODOS LOS RESULTADOS")
print("="*80)

# 1. Guardar el DataFrame original
df.to_csv('df_original.csv', index=False)

# 2. Guardar todas las particiones
for nombre, partition_df in particiones.items():
    if nombre != 'df_original': # Ya lo guardamos
        filename = f'{nombre}_partition.csv'
        partition_df.to_csv(filename, index=False)
        print(f"Partición guardada en '{filename}'")

# 3. Guardar las estadísticas
df_stats.to_csv('estadisticas_particiones.csv')
print("Tabla de estadísticas guardada en 'estadisticas_particiones.csv'")